<div style="font-weight:bold;"><center><h1>Projet IA - De la fast fashion à la fashion éthique</h1></center></div>
<center><h2>La place des enfants<br>Zoom sur l'enfant des pays producteurs, et sur l'enfant consommateur </h2></center>
<center><h3>Projet présenté par Shabboo Aleagha et Ophélie Engasser - MI IA Elearning - mai 2023</h3></center>

# **Plan**

*   **Axe 1. La fast fashion et le droit des enfants des pays de fabrication**
*   **Axe 2. La fast fashion et la santé des enfants consommateurs**
*   **Axe 3. Mise en perspective vers des solutions possibles faisant appel à l'intelligence artificielle**

Les axes 1 et 2 seront étudiés à l'aide d'indicateurs extraits de données brutes pour aboutir à des résultats mesurables et visualisables, dans l’optique d’ouvrir une réflexion sur la mise en œuvre un modèle d’apprentissage automatique (axe 3).    

# **Indicateurs retenus**

**Axe 1. Fast Fashion et enfants des pays de fabrication :**

**1.1 Indicateurs définis à partir des données récupérées d'un PDF sur le site Vertbaudet :**</br> 
Nombre de vêtements fabriqués par pays (en unités)</br>
Empreinte carbone de chaque combinaison d'étapes (tissage - teinture - confection) (en kg/eqCO2)</br>
Empreinte eau de chaque combinaison d'étapes (tissage - teinture - confection) (en Litres)</br>
**1.2 Indicateurs définis à partir des données scrapées sur le site de la Banque Mondiale (données de 2020 indiquées par pays de fabrication) :**
Taux d'évolution sur 10 ans de la valeur ajoutée de l'industrie manufacturière (en %)</br>
Valeur ajoutée de l'industrie textile (en billions de dollars)</br>
Taux de travail infantile (en %)</br>
Répartition des enfants travailleurs selon une activité unique ou combinée avec l'école (en %)</br>
Taux de pauvreté (< 6.85$ par jour) (en %)</br>
Volumes d'eau disponible vs. prélevée annuellement (en billions de m3)</br>
Taux d'accès aux énergies propres pour cuisiner (en %)</br>
Prévalences de la malnutrition et du retard de croissance (en %)</br>
Espérance de vie (en années)</br>
Taux de mortalité infantile (< 5 ans) (en %)</br></br>
**Axe 2. Fast Fashion et enfants consommateurs :**

**Indicateurs définis à partir des données scrapées sur le site Vertbaudet :**</br>
Répartition des matières textiles par collection (en %)</br> 
Score de toxicité des vêtements (entre 0 et 5 sur une échelle de Likert)</br></br>

Ce notebook présente linéairement les démarches inhérentes à chaque indicateur, ainsi que le résultat obtenu. Chaque décision est commentée en regard des limites rencontrées.

In [ ]:
# installation des librairies
!pip install tabula-py
!pip install haversine

In [ ]:
# import des librairies

# pour le scraping des données
import tabula
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from time import time
from IPython.display import clear_output
from warnings import warn
warn('Attention')

# pour les regex
import re

# pour l'exportation de fichiers
from google.colab import files

# pour la manipulation des données
import pandas as pd
import numpy as np
import math
import ast

# pour le calcul de distances
from haversine import haversine, Unit

# pour la visualisation des données
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# **Axe 1. La fast fashion et les droits des enfants des pays de fabrication**

## 1.1. Les pays d'origine de fabrication des vêtements de la marque Vertbaudet et leur impact environnemental

### Récupération des données sur l'origine de fabrication des vêtements du site Vertbaudet et calcul d'un score d'empreinte carbone et d'empreinte eau

Les renseignements sur les pays d'origine de fabrication du site Vertbaudet ne sont pas accessibles directement sur le lien de chaque article, mais sont regroupés dans un tableau disponible en PDF. Pour répondre à notre objectif, nous avons pu constituer un dataframe contenant chaque référence d'article associée à chaque pays selon l'étape de fabrication de l'article (il y a 3 étapes comme nous le verrons). Une ligne de ce dataframe correspond donc à un article. Les chaussures du site font également partie du dataset. Nous allons enrichir ce dataframe au fur et à mesure en y ajoutant des données d'intérêt concernant la distance parcourue par les articles, leur poids, afin d'étudier les impacts en terme de pollution (**empreinte carbone**) et de consommation d'eau (**empreinte eau**).

In [ ]:
# récupérons les données sur le tableau du site disponible en PDF
data = tabula.read_pdf('https://securemedia.vertbaudet.fr/Pdf/3-0-0/RSE/Tracabilite-produit-pays-textile-chaussure.pdf', pages='all') # récupération des 86 pages
df_countries = pd.DataFrame(np.concatenate(data), columns=['Référence article', 'Saison', 'Tissage', 'Teinture', 'Confection']) # concaténation des 86 pages en un dataframe

# nettoyons le dataset
df_countries = df_countries.drop(index=[0, 1, 2506, 3454, 3456, 3460, 3465, 3898, 3899]) # suppression des lignes contenant des données aberrantes
cols_to_apply = ['Tissage', 'Teinture', 'Confection'] 
df_countries[cols_to_apply] = df_countries[cols_to_apply].apply(lambda x: x.str.capitalize()) # harmonisation de l'écriture des noms de pays
df_countries

,Référence article,Saison,Tissage,Teinture,Confection
2,1002068,AH23,Chine,Chine,Chine
3,1002075,AH23,Chine,Chine,Chine
4,1002095,PE23,Bangladesh,Bangladesh,Bangladesh
5,1002099,PE23,Chine,Chine,Chine
6,1002099,PE23,Chine,Chine,Chine
...,...,...,...,...,...
4152,10543446409,PE23,Chine,Chine,Chine
4153,10639066390,AH23,Chine,Chine,Chine
4154,10639076630,PE23,Chine,Chine,Chine
4155,10639086400,AH23,Chine,Chine,Chine


Nous observons sur ce dataset que les 3 colonnes contenant les pays de fabrication constituent 3 étapes de fabrication : l'étape 1 étant le **tissage/tricotage** (nous l'appellerons tissage par simplicité), l'étape 2 la **teinture/impression** (nous l'appellerons teinture) et l'étape 3 la **confection**. Nous considérerons ces 3 étapes comme autant de chemin parcouru par les vêtements avant expédition vers les sites de vente Français. Cela nous permettra de calculer un indicateur de distance à partir duquel nous pourrons calculer un score d'**empreinte carbone**. Nous justifierons l'usage de cette mesure plutôt que celle d'émission de CO2. Nous allons également nous intéresser à l'**empreinte eau**, c'est-à-dire un indicateur de consommation d'eau que nous allons calculer à partir de nos données. Cela nous permettra d'enquêter sur l'environnement dans lequel évoluent les enfants vivant dans les pays de fabrication.

In [ ]:
# nous souhaitons obtenir le pays le plus représenté (dans le dataset, indépendamment des étapes) : 
steps = df_countries.iloc[:, -3:] # sélection des 3 dernières colonnes du dataframe
occurrences_pays = steps.stack().value_counts() # count des fréquences d'apparition de chaque pays (stack() empile les pays)
occurrences_pays

Bangladesh    5047
Chine         4269
Inde          2081
Pakistan       570
Portugal       168
Turquie        102
Vietnam         81
Maroc           42
Allemagne       32
Italie          24
Pologne         16
France           9
Brésil           3
dtype: int64

In [ ]:
# visualisons sur un pie chart la répartition des vêtements
fig_steps = go.Figure(data=[go.Pie(labels=occurrences_pays.index, values=occurrences_pays.values, hole=.5)])
fig_steps.update_layout(height=500, width=1200, title_text='Répartition du nombre de vêtements fabriqués selon les pays', piecolorway=['#b3e2cd', '#fdccac', '#cbd5e8', '#f4cae4', '#e6f5c9', '#fff2ae', '#f1e2cc', '#cccccc', '#decbe4', '#f2f2f2', '#ffffb3', '#9eb9f3', '#d9d9d9'])
fig_steps.show()

Nous constatons que les pays les plus représentés dans les étapes de fabrication des vêtements sont : le **Bangladesh** (env. 40% des vêtements fabriqués), la **Chine** (env. 35% des vêtements), l'**Inde** (env. 17% des vêtements) et le **Pakistan** (env. 5% des vêtements). Puis arrivent (entre 0.5 et 1.35%) : le **Portugal**, la **Turquie** et le **Vietnam**. Parmi ces pays, certains figurent dans la liste des pays qui ont recours au travail des enfants pour la culture du coton (Brésil, Turquie, Chine, Inde, Pakistan) ou l'industrie du textile (Bangladesh, Chine, Inde) ou de la chaussure (Brésil).

In [ ]:
# repérons les pays sur une carte afin de visualiser la distance parcourue par les vêtements jusqu'à la France
# pour cela créons un dataframe contenant les geopoints des pays (latitudes, longitudes)
data = {'Pays': ['Bangladesh', 'Chine', 'Inde', 'Pakistan', 'Portugal', 'Turquie', 'Vietnam', 'Maroc', 'Allemagne', 'Italie', 'Pologne', 'France', 'Brésil'],
        'Latitude': [24.476929, 35.000074, 22.351115, 30.330840, 39.662165, 38.959759, 15.926666, 31.172821, 51.163818, 42.638426, 52.215933, 46.227638, -10.333333],
        'Longitude': [90.293441, 104.999927, 78.667743, 71.247499, -8.135352, 34.924965, 107.965086, -7.336248, 10.447831, 12.674297, 19.134422, 2.213749, -53.200000],
        'Nb vêtements': occurrences_pays.values}
df_map = pd.DataFrame(data)

# carte
map = go.Figure(go.Scattergeo(lon = df_map['Longitude'], lat = df_map['Latitude'], mode = 'markers+text', text = df_map['Pays'], textposition = 'bottom center', marker = dict(size = 10, color = df_map['Nb vêtements'], colorscale = 'Portland', colorbar_title = "Nb de vêtements")))
map.update_layout(title = 'Pays de fabrication des vêtements Vertbaudet')
map.update_geos(scope='world', showcountries=True, landcolor='#D4EFDF', coastlinecolor='gray', countrycolor='gray')
map.show()

A présent nous souhaitons nous intéresser aux différentes étapes qui sont parcourues par les vêtements dans leur chaîne de fabrication. Il s'agit comme nous l'avons mentionné des 3 colonnes du dataframe récupéré sur le site Vertbaudet. Cela nous permettra de calculer un score d'empreinte carbone, qui sera fonction des distances entre chaque pays de la dernière étape et la France, mais aussi des distances entre chaque étape. Pour cela, il est nécessaire de mesurer toutes les combinaisons d'étapes.

In [ ]:
# observons la combinaison des étapes de fabrication : pour cela, utilisons une table pivot qui va arranger les données en les agrégeant (count) selon les 3 étapes considérées comme multi-index
table = pd.pivot_table(df_countries, index=['Tissage', 'Teinture', 'Confection'], values='Référence article', aggfunc='count') # création de la table pivot
table = table.sort_values('Référence article', ascending=False) # tri décroissant
table['Pourcentage'] = table['Référence article'] / table['Référence article'].sum() * 100 # ajout d'une colonne contenant le % de chaque combinaison d'étapes pour une meilleure visibilité des chiffres
table # 4148 articles au total

,,,Référence article,Pourcentage
Tissage,Teinture,Confection,,
Bangladesh,Bangladesh,Bangladesh,1648,39.729990
Chine,Chine,Chine,1355,32.666345
Inde,Inde,Inde,680,16.393443
Pakistan,Pakistan,Pakistan,190,4.580521
Portugal,Portugal,Portugal,56,1.350048
Chine,Chine,Bangladesh,51,1.229508
Inde,Chine,Chine,39,0.940212
Turquie,Turquie,Turquie,34,0.819672
Vietnam,Vietnam,Vietnam,27,0.650916


In [ ]:
# visualisons le résultat sur un bar chart horizontal
comb_steps = go.Figure(go.Bar(x=table['Référence article'], y=table.index.map(lambda x: ' - '.join(x)), text=table['Référence article'], textposition='outside', orientation='h', marker_color='pink'))
comb_steps.update_layout(title="Les différentes combinaisons d'étapes de fabrication", plot_bgcolor="rgba(0,0,0,0)", xaxis_title='Nombre de vêtements', bargap=0.15, margin=dict(l=150, r=50, t=50, b=50))
comb_steps.update_layout(yaxis_categoryorder='array', yaxis={'categoryarray': table.sort_values('Pourcentage', ascending=True).index.get_level_values(0) + " - " + table.sort_values('Pourcentage', ascending=True).index.get_level_values(1) + " - " + table.sort_values('Pourcentage', ascending=True).index.get_level_values(2)})
comb_steps.show()

Calculons à présent **les distances** : notre raisonnement est le suivant : nous calculons, à l'aide de la fonction haversine() présente dans une bibliothèque Python, la distance depuis le pays de la 3e étape jusqu'à la France, en ajoutant les étapes intermédiaires. Nous partons du principe que les vêtements ne font qu'un voyage du pays de confection vers la France, par conséquent si le pays est déjà comptabilisé pour la phase 3 dans un calcul précédent, nous ne considérons pas cela comme une distance supplémentaire vers la France.</br>
**La fonction haversine()** se base sur la formule du même nom, qui est utilisée pour calculer la distance entre deux points sur une sphère à partir de leurs **geopoints** (coordonnées de longitude et de latitude). Elle calcule la distance la plus courte entre deux points à la surface d'un objet sphérique, comme la Terre.</br>
Dans le dataframe utilisé, du fait de l'absence de schéma constant dans les étapes, il ne nous a pas été possible d'automatiser ces calculs à l'intérieur d'une boucle dans notre programme. Un traitement de chaque ligne au cas par cas nous a paru ici plus optimal.

In [ ]:
# calculons les distances
coord_france = (46.603354,1.8883335)
dist_1 = haversine((24.4769288,90.2934413), coord_france, unit=Unit.KILOMETERS) # distance pour la 1re combinaison d'étapes : Bangladesh->France
dist_2 = haversine((35.000074,104.999927), coord_france, unit=Unit.KILOMETERS) # Chine->France
dist_3 = haversine((22.3511148,78.6677428), coord_france, unit=Unit.KILOMETERS) # Inde->France
dist_4 = haversine((30.3308401,71.247499), coord_france, unit=Unit.KILOMETERS) # Pakistan->France
dist_5 = haversine((39.6621648,-8.1353519), coord_france, unit=Unit.KILOMETERS) # Portugal->France
dist_6 = haversine((35.000074,104.999927), (24.4769288,90.2934413), unit=Unit.KILOMETERS) # Chine->Bangladesh
dist_7 = haversine((22.3511148,78.6677428), (35.000074,104.999927), unit=Unit.KILOMETERS) # Inde->Chine
dist_8 = haversine((38.9597594,34.9249653), coord_france, unit=Unit.KILOMETERS) # Turquie->France
dist_9 = haversine((15.9266657,107.9650855), coord_france, unit=Unit.KILOMETERS) # Vietnam->France
dist_10 = haversine((51.1638175,10.4478313), (52.215933,19.134422), unit=Unit.KILOMETERS) + haversine((52.215933,19.134422), coord_france, unit=Unit.KILOMETERS)  # Allemagne->Pologne->France
dist_11 = haversine((35.000074,104.999927), (24.4769288,90.2934413), unit=Unit.KILOMETERS) # Chine->Bangladesh
dist_12 = haversine((31.1728205,-7.3362482), coord_france, unit=Unit.KILOMETERS) # Maroc->France
dist_13 = haversine((42.6384261,12.674297), coord_france, unit=Unit.KILOMETERS) # Italie->France
dist_14 = haversine((35.000074,104.999927), (24.4769288,90.2934413), unit=Unit.KILOMETERS) # Chine->Bangladesh
dist_15 = haversine((-10.3333333,-53.2), (24.4769288,90.2934413), unit=Unit.KILOMETERS) # Brésil->Bangladesh
dist_16 = haversine(coord_france, coord_france, unit=Unit.KILOMETERS) # France->France
dist_17 = haversine((35.000074,104.999927), (22.3511148,78.6677428), unit=Unit.KILOMETERS) # Chine->Inde
distances = [dist_1, dist_2, dist_3, dist_4, dist_5, dist_6, dist_7, dist_8, dist_9, dist_10, dist_11, dist_12, dist_13, dist_14, dist_15, dist_16, dist_17]
table = table.assign(Distances=distances)

A présent que nous avons les distances, nous pouvons nous pencher sur l'**empreinte carbone**. Pour cela il nous faut ajouter **les poids des vêtements**, nécessaires pour mesurer l'empreinte carbone. Nous considérerons un poids moyen par vêtement, étant donné que nous n'avons pas pu récupérer les nom/types de vêtements. Ici, nous allons fixer 250g comme poids moyen pour un vêtement d'enfant, en nous basant sur quelques recherches sur des sites spécialisés. 

```
# poids = nombre de vêtements * 0.25 kg
```



In [ ]:
# ajoutons les poids dans le dataframe
weights = table['Référence article'] * 0.25
table = table.assign(Poids=weights)

Nous choisissons dans cette étude de mesurer l'**empreinte carbone (ou émission de gaz à effet de serre)** et non le taux de CO2 émis car l'empreinte carbone nous paraît plus adaptée, complète et précise, puisqu'elle englobe l'ensemble des gaz émis pour une production ou une activité. Certains, comme le méthane, ont un potentiel de réchauffement climatique 25x plus important que le CO2 [[1]](https://greenly.earth/fr-fr/blog/actualites-ecologie/empreinte-carbone-comparatif-transports). L'empreinte carbone est généralement mesurée en tonnes équivalent CO2 (t/eqCO2), mais il est possible aussi de le faire en kg, comme nous allons le faire ici.
</br>
Notre calcul sera décomposé en deux indicateurs : le coût du **trajet en avion** pour le transport des vêtements, et le **coût de l'activité de production**.
```
# au total : 
# empreinte carbone totale = distance * empreinte liée au trajet + poids vêtements * empreinte liée à la production
```
**Pour le trajet** : un avion rejette en moyenne 360 g équivalents CO2 pour 1 km [[2]](https://consommerresponsable.com/empreinte-carbone-mode-de-transport-plus-polluant/). 

```
# empreinte carbone trajet = distance en km * 0.36 kg equiv. CO2
```
**Pour la production** : en 2019, 1 kg de textile consommé en France (dans un vêtement ou un linge de maison grand public) génère 54 kg d’équivalent CO2 (avec une production importée à 95,7%) [[3]](https://www.franceindustrie.org/wp-franceindustrie/wp-content/uploads/2021/02/Synthese-Etude-UIT-Empreinte-carbone-280121.pdf). 

```
# empreinte carbone production = poids en kg * 54 kg equiv. CO2
```






In [ ]:
# ajoutons l'empreinte carbone (en kg équivalent CO2) (kg/eqCO2) selon la distance et l'activité de production du vêtement
carbon_footprint = table['Distances'] * 0.36 + table['Poids'] * 54
table = table.assign(Empreinte_carbone=carbon_footprint)

Calculons maintenant un score d'empreinte eau, soit la consommation d'eau demandée pour la production des vêtements de la marque présents dans ce dataset.
Notre calcul sera décomposé de la manière suivante, respectant les étapes parcourues par un vêtement pour sa fabrication : étape de **culture des matières premières** lorsqu'elle sont naturelles ou artificielles, étape de **teinture** et étape de **lavage** dans les foyers.

```
# au total :      
# empreinte eau totale = empreinte liée à la culture + empreinte liée à la teinture + empreinte liée au lavage
```



**Pour la culture** : nous n'avons pas accès dans ce dataset aux matières premières, mais nous allons nous baser sur un chiffre largement diffusé, qui constitue selon nous une moyenne, un référentiel sur lequel nous allons nous appuyer. En effet, il faudrait environ 2700L d'eau pour un t-shirt (équivaut à 70 douches) [[4]](https://engagements.decathlon.fr/pourquoi-faut-il-tant-deau-pour-fabriquer-un-vetement). Nous prendrons donc 2500L comme référence, puisqu'il s'agit d'un vêtement moyen d'enfant.

```
# empreinte eau culture = nb d'articles * 2500L
```
**Pour la teinture** : il faut 100 à 150L d'eau pour 1 kg de textile teint [[4]](https://engagements.decathlon.fr/pourquoi-faut-il-tant-deau-pour-fabriquer-un-vetement). Ici nous prendrons donc la moyenne 125L.

```
# empreinte eau teinture = poids total en kg * 125L
```
**Pour le lavage** : cet article [[5]](https://particuliers.engie.fr/economies-energie/conseils-economies-energie/conseils-calcul-consommation/comment-calculer-la-consommation-de-votre-lave-linge.html) nous fournit des références moyennes de consommation d'eau pour une machine à laver : la moyenne haute pour une machine grande capacité serait de 65-70L ; la moyenne basse pour une machine petite capacité serait de 45-50L ; nous faisons donc la moyenne 57.5L. Un autre article [[6]](https://www.ubaldi.com/guides/lave-linge/caracteristiques-principales--gup208.php) nous donne la fourchette des capacités des machines à laver entre 5 et 12 kg, nous prenons la moyenne 8.5 kg. On divise la consommation par 8.5 pour obtenir la consommation d'eau de 1 kg de vêtements et cela constitue notre consommation d'eau pour cette étape, ce qui fait 6.75L.
```
# empreinte eau lavage = poids total en kg * 6.75L
```

In [ ]:
# ajoutons l'empreinte eau (en L)
water_footprint = table['Référence article'] * 2500 + table['Poids'] * 125 + + table['Poids'] * 6.75
table = table.assign(Empreinte_eau=water_footprint)
table.round(2)

,,,Référence article,Pourcentage,Distances,Poids,Empreinte_carbone,Empreinte_eau
Tissage,Teinture,Confection,,,,,,
Bangladesh,Bangladesh,Bangladesh,1648,39.73,7942.69,412.00,25107.37,4174281.00
Chine,Chine,Chine,1355,32.67,8139.02,338.75,21222.55,3432130.31
Inde,Inde,Inde,680,16.39,7234.55,170.00,11784.44,1722397.50
Pakistan,Pakistan,Pakistan,190,4.58,6097.06,47.50,4759.94,481258.12
Portugal,Portugal,Portugal,56,1.35,1119.64,14.00,1159.07,141844.50
Chine,Chine,Bangladesh,51,1.23,1836.13,12.75,1349.51,129179.81
Inde,Chine,Chine,39,0.94,2915.33,9.75,1576.02,98784.56
Turquie,Turquie,Turquie,34,0.82,2803.41,8.50,1468.23,86119.88
Vietnam,Vietnam,Vietnam,27,0.65,9902.87,6.75,3929.53,68389.31


In [ ]:
# visualisons l'empreinte carbone sur un bar chart horizontal
compare_carbon_footprint = go.Figure(go.Bar(x=table['Empreinte_carbone'], y=table.index.map(lambda x: ' - '.join(x)), text=table['Empreinte_carbone'].round(2), textposition='outside', orientation='h', marker_color='lightgray'))
compare_carbon_footprint.update_layout(title="Empreinte carbone liée à la distance et à l'activité de production des vêtements", plot_bgcolor="rgba(0,0,0,0)", xaxis_title='kg/eqCO2', bargap=0.15, margin=dict(l=150, r=50, t=50, b=50))
compare_carbon_footprint.update_layout(yaxis_categoryorder='array', yaxis={'categoryarray': table.sort_values('Empreinte_carbone', ascending=True).index.get_level_values(0) + " - " + table.sort_values('Empreinte_carbone', ascending=True).index.get_level_values(1) + " - " + table.sort_values('Empreinte_carbone', ascending=True).index.get_level_values(2)})
compare_carbon_footprint.show()

In [ ]:
# voyons à quoi ressemble la distribution de cette variable empreinte carbone
table['Empreinte_carbone'].describe()

count       17.000000
mean      4888.153052
std       7486.207907
min         40.500000
25%        877.006562
50%       1349.506562
75%       4759.941865
max      25107.367343
Name: Empreinte_carbone, dtype: float64

La médiane est de **1349.51 kg/eqCO2** ce qui signifie que la moitié des combinaisons ont un score en dessous de cette valeur. Nous préférons retenir la médiane, la moyenne étant à notre sens trop influencée par les valeurs extrèmes (outliers), comme nous allons le voir dans le boxplot.</br>
Nous observons aussi que 50% de la distribution présente un score entre **877 et 4760 kg/eqCO2** (l'intervalle inter-quartile), ce qui est loin, donc, du maximum qui est de **25107 kg/eqCO2**.

In [ ]:
# visualisons la distribution sur un boxplot et un violin plot
boxplot_carbon_footprint = px.box(table, y=table['Empreinte_carbone'])
violin_carbon_footprint = px.violin(table, y='Empreinte_carbone', box=True, points='all', title="Distribution de l'empreinte carbone")
boxplot_carbon_footprint.update_traces(boxmean=True, marker=dict(color='lightgray'))
violin_carbon_footprint.update_traces(meanline_visible=True, meanline_color='red', marker=dict(color='lightgray'), width=0.5, side='both')

fig_carbone = make_subplots(rows=1, cols=2)

fig_carbone.add_trace(boxplot_carbon_footprint.data[0], row=1, col=1)
fig_carbone.add_trace(violin_carbon_footprint.data[0], row=1, col=2)

fig_carbone.update_layout(title="Distribution de l'empreinte carbone", plot_bgcolor="rgba(0,0,0,0)", yaxis_title='kg/eqCO2', height=500, width=1350, showlegend=False)
fig_carbone.update_xaxes(title_text="Box plot", row=1, col=1)
fig_carbone.update_xaxes(title_text="Violin plot", row=1, col=2)
fig_carbone.show()

Sur le boxplot, nous voyons que la distribution est très écrasée, du fait d'une forte représentation des valeurs dans l'intervalle inter-quartile, ce qui se voit nettement également au niveau de la courbe de densité du violin plot. Les scores pour le **Bangladesh**, la **Chine**, et l'**Inde** surpassent très largement la distribution des empreintes carbones de toutes les étapes de production, au point d'être considérées comme des outliers dans le boxplot. Cela donne l'aspect allongé du violin plot sur les valeurs élevées.</br></br>
Pour nous donner un ordre de grandeur, la fabrication au Bangladesh, soit 25107 kg/eqCO2 équivaut à :</br>
- **100028 km réalisés avec le gasoil d'une voiture diesel classique** (en partant du principe qu'une voiture diesel de base consomme 0.251 kg de CO2/km [[7]](https://www.fournisseurs-electricite.com/guides/environnement/bilan-carbone/voiture)).
</br>
- **La consommation moyenne de 3 Français moyens en 1 an** (soit un petit ménage) (en effet, l'empreinte carbone d'un Français en 1 an est de 8200 kg de CO2 [[8]](https://www.notre-environnement.gouv.fr/actualites/breves/article/que-represente-l-empreinte-carbone-annuelle-d-un-francais)).
</br>
- **La capacité d'absorption de 5 arbres en 1 an** (sachant qu'un arbre absorbe environ 5000 kg eq/CO2 [[9]](https://www.terre-du-futur.fr/combien-de-co2-absorbe-un-arbre/)). 
</br>
- **25 vols aller/retour entre Paris et New-York** (un vol équivaut à 1000 kg eq/CO2, selon la source précédente [[9]](https://www.terre-du-futur.fr/combien-de-co2-absorbe-un-arbre/)).

In [ ]:
# visualisons l'empreinte eau sur un bar chart horizontal
compare_water_footprint = go.Figure(go.Bar(x=table['Empreinte_eau'], y=table.index.map(lambda x: ' - '.join(x)), text=table['Empreinte_eau'].round(2), textposition='outside', orientation='h', marker_color='lightblue'))
compare_water_footprint.update_layout(title="Empreinte eau liée à la confection des vêtements", plot_bgcolor="rgba(0,0,0,0)", xaxis_title='Litres', bargap=0.15, margin=dict(l=150, r=50, t=50, b=50))
compare_water_footprint.update_layout(yaxis_categoryorder='array', yaxis={'categoryarray': table.sort_values('Empreinte_eau', ascending=True).index.get_level_values(0) + " - " + table.sort_values('Empreinte_eau', ascending=True).index.get_level_values(1) + " - " + table.sort_values('Empreinte_eau', ascending=True).index.get_level_values(2)})
compare_water_footprint.show()

Nous observons, sans surprise, que les 1ers pays les plus impactants sur la planète sont le **Bangladesh**, la **Chine**, l'**Inde** et le **Pakistan**. Pour faire transition avec notre partie suivante, nous souhaitons souligner que les enfants vivant dans ces pays de fabrication (Bangladesh, Chine, Inde, Pakistan), parfois obligés de travailler dans les usines, sont fortement impactés par cette industrie, leur droit à vivre dans un environnement sain est mis à mal, et nous souhaitons nous focaliser à présent sur les indicateurs liés à leur environnement et conditions de vie. 

Nous ne présentons pas la distribution de la variable 'empreinte eau' car, comme l'on peut s'y attendre, elle suit exactement la même forme que celle de l'empreinte carbone, avec en tête les 4 mêmes pays, le **Bangladesh**, la **Chine**, l'**Inde** et le **Pakistan**. Selon la source sur laquelle nous nous sommes basées pour notre calcul, la confection d'un t-shirt équivaut à 70 douches [[4]](https://engagements.decathlon.fr/pourquoi-faut-il-tant-deau-pour-fabriquer-un-vetement), nous avons voulu convertir nos données en équivalents-douches : pour le Bangladesh, par ex., la production pour Vertbaudet équivaut à **108222 douches**.



**Nous constatons l'ampleur de l'empreinte carbone et eau des 4 premiers pays producteurs de vêtements Vertbaudet :**     
1. **Bangladesh**
2. **Chine**
3. **Inde**
4. **Pakistan**

Voyons maintenant si une tendance similaire s'observe quant aux indicateurs que nous avons retenus pour cerner la manière dont les droits des enfants sont exercés, ou non, dans les pays de fabrication. Vérifions si ces 4 pays forment également un "cluster" concernant ces données. C'est ce que nous allons à présent aborder en croisant ce constat avec les données de la Banque Mondiale.

## 1.2. Croisement des pays d'origine de fabrication des vêtements avec les données sociales de la Banque Mondiale

A présent, nous souhaitons poursuivre nos investigations en nous concentrant sur ce qu'il se passe au niveau des populations, et donc pour les enfants, dans les pays de fabrication : quelles sont les conditions de vie, de travail forcé, d'accès aux ressources de vie comme l'eau, l'électricité, d'accès au confort et à l'éducation, quelles sont les conditions de santé et la mortalité ?</br>
D'un point de vue méthodologique, nous allons scraper les données de la Banque Mondiale, qui sont présentées sur le site sous forme de table organisées en thématiques. Nous allons récupérer les tables les plus en lien avec notre problématique, les concaténer pour en faire un dataframe entier, à partir duquel nous travaillerons. Il est à noter que nous ne présenterons pas toutes les variables, un travail exploratoire ayant été réalisé pour ne retenir que les variables les plus informatives dans un soucis de visualisation et de compréhension de notre propos.</br> Notons encore, d'un point de vue technique, que chaque table du site de la Banque Mondiale se présente avec des balises 'td' qui portent toutes le même nom de classe 'spacer', il nous a donc fallu récupérer toutes les cellules de chaque table en même temps, et ensuite diviser les listes renvoyées en sous listes pour constituer un dataframe. Le nombre de cellules de chaque table choisie étant variable, nous n'avons pas pu automatiser le processus pour récupérer toutes les tables en même temps. Ainsi, nous avons scrapé chaque table, que nous avons concaténé par la suite.

### Scraping des données de la Banque Mondiale

Sur le site de la Banque Mondiale, nous nous intéressons à 7 indicateurs qui sont chacun répertoriés dans une table :</br>
1. Structure de l'industrie manufacturière
2. Travail des enfants et accès à l'éducation
3. Pauvreté
4. Accès à l'eau
5. Accès à l'électricité
6. Malnutrition et troubles associés
7. Mortalité
 
Comme nous l'avons mentionné, la difficulté est d'automatiser un processus de scraping, étant donné que chaque table comporte un nombre de lignes différent et des index de pays différents. La meilleure option que nous avons retenue, voire la seule, bien qu'elle ne soit pas optimale, nous permettra d'obtenir un dataframe complet, en scrapant toutes les tables séparément et en les concaténant comme nous ferions une jointure SQL.</br>
Enfin, concernant l'en-tête des tables, elles sont assez complexes car multilignes, avec des cellules imbriquées. Pas d'automatisation possible non plus à ce niveau, excepté pour la table sur le travail des enfants. Le nom des colonnes sera donc ajouté manuellement pour une meilleure lisibilité du dataframe final. Chaque variable sera décrite avec son unité de mesure.


In [ ]:
# process : 
# - récupération de toutes les cellules à la suite et ajout à une liste
# - transformation de la liste en liste de n listes correspondant à chaque pays x nb de variables et stockage dans un dataframe
# - pour child_labor : extraction du header par le scraping

In [ ]:
# ----- 1. manufacturing -----
url= 'http://wdi.worldbank.org/table/4.3#'
page = requests.get(url)
soup =  BeautifulSoup(page.text,'html.parser')
conteneur = soup.find_all('table', class_='indicators-table')[1] # extraire le corps (les lignes) de la table
tableau_cellules = conteneur.find_all('div', class_='spacer') # extraire toutes les cellules de la table
cellules = []
for i in tableau_cellules:
  cellules.append(i.text)

# transformer la liste en liste de 214 listes correspondant à chaque pays x 13 variables et stocker cela dans un dataframe
size = 13
sub_lists = [cellules[idx: idx+size] for idx in range(0, len(cellules), size)]
df_manufacturing = pd.DataFrame(sub_lists, columns = ["Country", 
                                                      "Val. ajoutée - 2010 (b.$)", # valeur ajoutée de l'industrie manufacturière (en billions de $)
                                                      "Val. ajoutée - 2020 (b.$)", # idem en 2020
                                                      "Alimentation, boisson, tabac - 2010 (en %)", # en % de la valeur ajoutée totale
                                                      "Alimentation, boisson, tabac - 2020 (en %)", # idem en 2020
                                                      "Textiles, vêtements - 2010 (en %)", # en % de la valeur ajoutée totale
                                                      "Textiles, vêtements - 2020 (en %)", # idem en 2020
                                                      "Machines, mat. de transport - 2010 (en %)", # en % de la valeur ajoutée totale
                                                      "Machines, mat. de transport - 2020 (en %)", # idem en 2020
                                                      "Prod. chimiques - 2010 (en %)", # en % de la valeur ajoutée totale
                                                      "Prod. chimiques - 2020 (en %)", # idem en 2020
                                                      "Autres - 2010 (en %)", # autres manufactures en 2010 en % de la valeur ajoutée totale
                                                      "Autres - 2020 (en %)" #  idem en 2020
                                                     ]).set_index('Country')

In [ ]:
# ----- 2. child_labor -----
url= 'http://wdi.worldbank.org/table/2.6'
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')
conteneur = soup.find_all('table', class_='indicators-table')[1] # extraire toutes les lignes de la table
tableau_cellules = conteneur.find_all('div', class_='spacer') # extraire toutes les cellules
cellules = []
for i in tableau_cellules:
  cellules.append(i.text)

# transformer la liste en liste de 214 listes correspondant à chaque pays x 13 variables et en faire un dataframe
size = 13
sub_lists = [cellules[idx: idx+size] for idx in range(0, len(cellules), size)]
df_child_labor = pd.DataFrame(sub_lists)

# extraire le header du tableau
header = soup.find_all('table', class_='indicators-table')[0]
head = header.find_all('th')[5:18]
text_head = [pt.get_text() for pt in head]

# remplacement de l'indice vide [0] par le pays et l'indice [1] par l'année pour correspondre au df
text_head[0] = "Country"
text_head[1] = 'Survey year'

# obtention du dataframe brut : les données sont représentées en % d'enfants âgés de 7 à 14 ans
df_child_labor = pd.DataFrame(sub_lists, columns = text_head).set_index('Country')

In [ ]:
# ----- 3. poverty -----
url= 'http://wdi.worldbank.org/table/1.2'
page = requests.get(url)
soup =  BeautifulSoup(page.text,'html.parser')
conteneur = soup.find_all('table', class_='indicators-table')[1] # extraire le corps (les lignes) de la table
tableau_cellules = conteneur.find_all('div', class_='spacer') # extraire toutes les cellules de la table
cellules = []
for i in tableau_cellules:
  cellules.append(i.text)

# transformer la liste en liste de 214 listes correspondant à chaque pays x 13 variables et stocker cela dans un dataframe
size = 5
sub_lists = [cellules[idx: idx+size] for idx in range(0, len(cellules), size)]
df_poverty = pd.DataFrame(sub_lists, columns = ["Country", 
                                               "Année de l'étude",
                                               "Pop. < 2.15$ / jour", # % de la pop. vivant avec moins de 2.15 $ par jour aux prix ajustés du pouvoir d'achat de 2017
                                               "Pop. < 3.65$ / jour", # % de la pop. vivant avec moins de 3.65 $ par jour
                                               "Pop. < 6.85$ / jour" # % de la pop. vivant avec moins de 6.85 $ par jour 
                                               ]).set_index('Country')

In [ ]:
# ----- 4. water -----
url= 'http://wdi.worldbank.org/table/3.5'
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')
conteneur = soup.find_all('table', class_='indicators-table')[1] # extraire toutes les lignes de la table
tableau_cellules = conteneur.find_all('div', class_='spacer') # extraire toutes les cellules
cellules = []
for i in tableau_cellules:
  cellules.append(i.text)

# transformer la liste en liste de 214 listes correspondant à chaque pays x 13 variables et stocker cela dans un dataframe
size = 11
sub_lists = [cellules[idx: idx+size] for idx in range(0, len(cellules), size)]
df_water = pd.DataFrame(sub_lists, columns = ["Country", 
                                        "Ressources internes en eau (b.m3)", #  flux de ressources en eau douce internes renouvelables (issus des fleuves et nappes phréatiques) (en billions de mètres cubes)
                                        "Ressources par habitant (m3)", #  idem par habitant (en mètres cubes)
                                        "Prélèvements annuels (b.m3)", # prélèvements annuels d'eau douce (au total : comprenant l'utilisation industrielle ou domestique) (en billions de mètres cubes) 
                                        "Ratio prélèvement / ressources (en %)", #  % total de prélèvements par rapport au total des ressources d'eau 
                                        "Prélèvements agriculture (en %)", # % prélèvements pour l'agriculture par rapport au total des prélèvements
                                        "Prélèvements industrie (en %)", # % prélèvements pour l'industrie par rapport au total des prélèvements 
                                        "Prélèvements domestiques (en %)", # % prélèvements pour l'usage domestique par rapport au total des prélèvements
                                        "Productivité de l'eau", # productivité de l'eau calculée comme le PIB à prix constants divisé par le prélèvement annuel total d'eau (mesuré en PIB en dollars par mètre cube) 
                                        "% pop. urbaine", # % de personnes (urbaines) utilisant au moins les services d'eau de base (eau potable provenant d'une source < 30 min de distance A/R, ou eau courante, forage...)
                                        "% pop. rurale" #  idem personnes rurales
                                        ]).set_index('Country')

In [ ]:
# ----- 5. energy -----
url= 'http://wdi.worldbank.org/table/3.13'
page = requests.get(url)
soup =  BeautifulSoup(page.text,'html.parser')
conteneur = soup.find_all('table', class_='indicators-table')[1] # extraire le corps (les lignes) de la table
tableau_cellules = conteneur.find_all('div', class_='spacer') # extraire toutes les cellules de la table
cellules = []
for i in tableau_cellules:
  cellules.append(i.text)

# transformer la liste en liste de 214 listes correspondant à chaque pays x 13 variables et stocker cela dans un dataframe
size = 11
sub_lists = [cellules[idx: idx+size] for idx in range(0, len(cellules), size)]
df_energy = pd.DataFrame(sub_lists, columns = ["Country", 
                                               "Accès à l'électricité - 2000 (en %)", # en % de la population
                                               "Accès à l'électricité - 2020 (en %)",
                                               "Accès à l'électricité - Pop. urbaine - 2020 (en %)", # en % de la population urbaine
                                               "Accès à l'électricité - Pop. rurale - 2020 (en %)", # en % de la population rurale
                                               "Accès aux énergies propres pour cuisiner - 2000 (en %)", # en % de la population
                                               "Accès aux énergies propres pour cuisiner - 2020 (en %)",
                                               "Consommation d'énergie renouvelable - 2000 (en %)", # en % de l'énergie totale consommée
                                               "Consommation d'énergie renouvelable - 2020 (en %)",
                                               "Production d'électricité renouvelable - 2000 (en %)", # en % de l'électricité totale produite
                                               "Production d'électricité renouvelable - 2015 (en %)",
                                               ]).set_index('Country')

In [ ]:
# ----- 6. malnutrition -----
url= 'http://wdi.worldbank.org/table/2.15'
page = requests.get(url)
soup =  BeautifulSoup(page.text,'html.parser')
conteneur = soup.find_all('table', class_='indicators-table')[1] # extraire le corps (les lignes) de la table
tableau_cellules = conteneur.find_all('div', class_='spacer') # extraire toutes les cellules de la table
cellules = []
for i in tableau_cellules:
  cellules.append(i.text)

# transformer la liste en liste de 214 listes correspondant à chaque pays x 13 variables et stocker cela dans un dataframe
size = 13
sub_lists = [cellules[idx: idx+size] for idx in range(0, len(cellules), size)]
df_nutrition = pd.DataFrame(sub_lists, columns = ["Country", 
                                                  "Prévalence de la malnutrition - 2001 (en %)", # en % de la population
                                                  "Prévalence de la malnutrition - 2020 (en %)",
                                                  "Prévalence du retard de croissance - 2000 (en %)", # en % des enfants < 5 ans
                                                  "Prévalence du retard de croissance - 2020 (en %)",
                                                  "Prévalence du surpoids - 2000 (en %)", # en % des enfants < 5 ans
                                                  "Prévalence du surpoids - 2020 (en %)",
                                                  "Prévalence de l'émaciation - Garçons - 2012/20 (en %)", # en % des garçons < 5 ans
                                                  "Prévalence de l'émaciation - Filles - 2012/20 (en %)", # en % des filles < 5 ans
                                                  "Prévalence de l'émaciation sévère - Garçons - 2012/20 (en %)", # idem
                                                  "Prévalence de l'émaciation sévère - Filles - 2012/20 (en %)",
                                                  "Prévalence de l'insuffisance pondérale - Garçons - 2012/20 (en %)",
                                                  "Prévalence de l'insuffisance pondérale - Filles - 2012/20 (en %)"
                                                 ]).set_index('Country')

In [ ]:
# ----- 7. mortality -----
url= 'http://wdi.worldbank.org/table/2.18#'
page = requests.get(url)
soup =  BeautifulSoup(page.text,'html.parser')
conteneur = soup.find_all('table', class_='indicators-table')[1] # extraire le corps (les lignes) de la table
tableau_cellules = conteneur.find_all('div', class_='spacer') # extraire toutes les cellules de la table
cellules = []
for i in tableau_cellules:
  cellules.append(i.text)

# transformer la liste en liste de 214 listes correspondant à chaque pays x 13 variables et stocker cela dans un dataframe
size = 13
sub_lists = [cellules[idx: idx+size] for idx in range(0, len(cellules), size)]
df_mortality = pd.DataFrame(sub_lists, columns = ["Country", 
                                                  "Espérance de vie - 1990", # en années
                                                  "Espérance de vie - 2020",
                                                  "Taux de mortalité néonatale - 1990 (en ‰)", # survenant avant 28 jours (pour 1000 naissances en vie)
                                                  "Taux de mortalité néonatale - 2020 (en ‰)",
                                                  "Taux de mortalité infantile - 1990 (en ‰)", # survenant avant 1 an (pour 1000 naissances en vie)
                                                  "Taux de mortalité infantile - 2020 (en ‰)",
                                                  "Taux de mortalité < 5 ans - 1990 (en ‰)", # survenant avant 5 ans (pour 1000 naissances en vie)
                                                  "Taux de mortalité < 5 ans - 2020 (en ‰)",
                                                  "Taux de mortalité < 5 ans - Garçons - 2020 (en ‰)", # idem
                                                  "Taux de mortalité < 5 ans - Filles - 2020 (en ‰)",
                                                  "Taux de mortalité adultes - Hommes - 2015/20 (en ‰)", # pour 1000 adultes hommes
                                                  "Taux de mortalité adultes - Femmes - 2015/20 (en ‰)" # pour 1000 adultes femmes
                                                 ]).set_index('Country')

In [ ]:
# concaténation de tous les dataframes en privilégiant une 'inner join' afin de faire correspondre tous les index (certains pays sont décalés)
df = pd.concat([df_manufacturing, df_child_labor, df_poverty, df_water, df_energy, df_nutrition, df_mortality], axis=1, join='inner')

# suppression des colonnes 'dates de l'étude' que nous n'exploiterons pas
df = df.drop(["Survey year", "Année de l'étude"], axis=1)
df.head(4)

,Val. ajoutée - 2010 (b.$),Val. ajoutée - 2020 (b.$),"Alimentation, boisson, tabac - 2010 (en %)","Alimentation, boisson, tabac - 2020 (en %)","Textiles, vêtements - 2010 (en %)","Textiles, vêtements - 2020 (en %)","Machines, mat. de transport - 2010 (en %)","Machines, mat. de transport - 2020 (en %)",Prod. chimiques - 2010 (en %),Prod. chimiques - 2020 (en %),...,Taux de mortalité néonatale - 1990 (en ‰),Taux de mortalité néonatale - 2020 (en ‰),Taux de mortalité infantile - 1990 (en ‰),Taux de mortalité infantile - 2020 (en ‰),Taux de mortalité < 5 ans - 1990 (en ‰),Taux de mortalité < 5 ans - 2020 (en ‰),Taux de mortalité < 5 ans - Garçons - 2020 (en ‰),Taux de mortalité < 5 ans - Filles - 2020 (en ‰),Taux de mortalité adultes - Hommes - 2015/20 (en ‰),Taux de mortalité adultes - Femmes - 2015/20 (en ‰)
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,2.0,1.5,..,..,..,..,..,..,..,..,...,74,35,121,45,179,58,61,54,319,210
Albania,0.7,0.9,15.7,19.6,25.6,36.4,2.0,2.4,4.6,1.8,...,13,7,35,8,41,9,10,9,115,56
Algeria,62.9,27.3,42.8,..,3.3,..,..,..,9.7,..,...,24,16,44,20,52,23,25,21,115,82
American Samoa,0.0,0.1,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [ ]:
# transformons les types object en float
for col in df.columns:
  df[col] = df[col].apply(pd.to_numeric, errors='coerce')

In [ ]:
# affichons le % de NaN dans le dataframe
df.isna().sum().sum() / df.size * 100

30.4739652870494

In [ ]:
# préparons un dataframe contenant uniquement les pays de fabrication, sur lesquels nous centrerons notre travail d'analyse
df_countries_selected = df.loc[["Bangladesh", "China", "India", "Pakistan", "Portugal", "Turkiye", "Vietnam", "Morocco", "Poland", "Brazil"],]
df_countries_selected = df_countries_selected.assign(NbArticles=occurrences_pays) # ajoutons la série de notre 1re partie : 'Référence article' soit le nb d'articles produits
df_countries_selected.head(4)

,Val. ajoutée - 2010 (b.$),Val. ajoutée - 2020 (b.$),"Alimentation, boisson, tabac - 2010 (en %)","Alimentation, boisson, tabac - 2020 (en %)","Textiles, vêtements - 2010 (en %)","Textiles, vêtements - 2020 (en %)","Machines, mat. de transport - 2010 (en %)","Machines, mat. de transport - 2020 (en %)",Prod. chimiques - 2010 (en %),Prod. chimiques - 2020 (en %),...,Taux de mortalité néonatale - 2020 (en ‰),Taux de mortalité infantile - 1990 (en ‰),Taux de mortalité infantile - 2020 (en ‰),Taux de mortalité < 5 ans - 1990 (en ‰),Taux de mortalité < 5 ans - 2020 (en ‰),Taux de mortalité < 5 ans - Garçons - 2020 (en ‰),Taux de mortalité < 5 ans - Filles - 2020 (en ‰),Taux de mortalité adultes - Hommes - 2015/20 (en ‰),Taux de mortalité adultes - Femmes - 2015/20 (en ‰),NbArticles
Country,,,,,,,,,,,,,,,,,,,,,
Bangladesh,18.6,77.0,NaN,15.4,NaN,56.8,NaN,3.6,NaN,3.1,...,17.0,101.0,24.0,146.0,29.0,31.0,27.0,156.0,117.0,5047.0
China,NaN,NaN,11.8,11.8,10.0,10.0,24.5,24.5,10.8,10.8,...,3.0,43.0,6.0,54.0,7.0,8.0,7.0,109.0,55.0,NaN
India,285.4,365.0,9.5,NaN,8.8,NaN,19.1,NaN,14.9,NaN,...,20.0,89.0,27.0,127.0,32.0,32.0,33.0,202.0,144.0,NaN
Pakistan,23.2,34.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,107.0,54.0,140.0,66.0,70.0,61.0,215.0,140.0,570.0


In [ ]:
# ajoutons des données qui n'ont pas été gérées par le scraping du fait d'un séparateur de milliers (pour les données sur l'eau)
df_countries_selected.loc["China", "Ressources internes en eau (b.m3)"] = 2812.9
df_countries_selected.loc["China", "Ressources par habitant (m3)"] = 1998
df_countries_selected.loc["India", "Ressources internes en eau (b.m3)"] = 1146
df_countries_selected.loc["India", "Ressources par habitant (m3)"] = 1045
df_countries_selected.loc["Brazil", "Ressources internes en eau (b.m3)"] = 5661
df_countries_selected.loc["Brazil", "Ressources par habitant (m3)"] = 26730

In [ ]:
# % de NaN dans le sous ensemble
df_countries_selected.isna().sum().sum() / df.size * 100

0.5767940979208585

Pour chaque indicateur, nous utiliserons le dataset contenant l'ensemble des pays, puis le dataset des pays de fabrication de la marque Vertbaudet pour observer les tendances.

### Zoom sur l'industrie manufacturière

Commençons par visualiser la manière dont se distribue **la valeur ajoutée de l'industrie manufacturière (qui comprend l'industrie textile)**, selon le type de secteur, et ce pour l'ensemble du dataset mondial. Pour cela, réalisons des boxplot pour chaque secteur. La valeur ajoutée est exprimée pour chaque secteur en % par rapport à la valeur ajoutée totale de l'industrie manufacturière.

In [ ]:
# visualisons les distributions sur des boxplots
boxplot_manufacturing = make_subplots(rows=1, cols=8, subplot_titles=("Alimentation, boisson, tabac", "Texiles, vêtements", "Machines, transport", "Produits chimiques", "Autres"))
boxplot_manufacturing.add_trace(go.Box(y=df["Alimentation, boisson, tabac - 2020 (en %)"], opacity=0.5), row=1, col=1)
boxplot_manufacturing.add_trace(go.Box(y=df["Textiles, vêtements - 2020 (en %)"], opacity=0.5), row=1, col=2)
boxplot_manufacturing.add_trace(go.Box(y=df["Machines, mat. de transport - 2020 (en %)"], opacity=0.5), row=1, col=3)
boxplot_manufacturing.add_trace(go.Box(y=df["Prod. chimiques - 2020 (en %)"], opacity=0.5), row=1, col=4)
boxplot_manufacturing.add_trace(go.Box(y=df["Autres - 2020 (en %)"], opacity=0.5), row=1, col=5)
boxplot_manufacturing.update_layout(title="Distributions de la valeur ajoutée mondiale de l'industrie manufacturière selon le type en 2020", showlegend=False, plot_bgcolor="rgba(0,0,0,0)"),
boxplot_manufacturing.update_layout(width=2200, yaxis_title="% par rapport à la val. ajoutée totale")
boxplot_manufacturing.update_xaxes(showticklabels=False)
boxplot_manufacturing.update_yaxes(range=[0, 100])
boxplot_manufacturing.show()

Nous observons que le % de valeur ajoutée pour l'industrie textile se trouve en-dessous des valeurs des autres types. Avec des outliers très supérieurs (allant jusqu'à **56.8%**) par rapport à la médiane (**3.6%**). Un premier constat intéressant est que, pour les mêmes indicateurs en 2010, la médiane était à **4.2%** et la valeur maximale à **34%**. Cela signifie que les écarts se sont creusés entre les valeurs centrales et les valeurs extrèmes. Le pays avec le maximum de part de valeur ajoutée en 2020 dans le textile n'est autre que le **Bangladesh**, le pays que nous avons vu apparaître en première position dans notre dataset des pays de fabrication de la marque Vertbaudet. Sa valeur avoisinne les valeurs ajoutées maximales pour les autres secteurs présentés. 

In [ ]:
# cherchons les 3 autres pays 'outliers', qui ne figurent pas parmi nos pays de fabrication
country_outlier_2 = df.index[df["Textiles, vêtements - 2020 (en %)"] == 36.4][0]
country_outlier_3 = df.index[df["Textiles, vêtements - 2020 (en %)"] == 31.6][0]
country_outlier_4 = df.index[df["Textiles, vêtements - 2020 (en %)"] == 21.3][0]
print(country_outlier_2, country_outlier_3, country_outlier_4)

Albania Mauritius Nicaragua


L'industrie du textile, comme on peut le voir sur les boxplot, est celle qui présente le plus, avec l'industrie des produits chimiques, de deltas importants entre les outliers et les valeurs de la boîte.</br></br>
Il va être intéressant à présent de nous pencher sur le dataset des pays de fabrication, afin de pouvoir effectuer un comparatif. Observons dans un 1er temps **l'évolution de la valeur ajoutée totale de l'industrie manufacturière entre 2010 et 2020**. La valeur ajoutée est ici mesurée en billions de dollars.

In [ ]:
# réalisons un graphique d'évolution de la valeur ajoutée entre 2010 et 2020
traces = []
for country in df_countries_selected.index:
    trace = go.Scatter(x=[2010, 2020], y=[df_countries_selected.loc[country, "Val. ajoutée - 2010 (b.$)"], df_countries_selected.loc[country, "Val. ajoutée - 2020 (b.$)"]], mode='lines+markers', name=country, text=[f"{country}: {val:,.2f} b.$" for val in [df_countries_selected.loc[country, "Val. ajoutée - 2010 (b.$)"], df_countries_selected.loc[country, "Val. ajoutée - 2020 (b.$)"]]], textposition='top center', hoverinfo='text', line=dict(width=4))
    traces.append(trace)
fig_evolution = go.Figure(data=traces)
fig_evolution.update_layout(title="Valeur ajoutée de l'industrie manufacturière en 2010 et 2020, tous secteurs confondus", xaxis=dict(showgrid=True, gridcolor='lightgray', title='Année', tickvals=[2010, 2020]), yaxis=dict(showgrid=True, gridcolor='lightgray', title="Val. ajoutée en billions de $"), plot_bgcolor="rgba(0,0,0,0)", hovermode='closest')
fig_evolution.show()

**Concernant la valeur ajoutée à l'intérieur des pays** : nous constatons que c'est le **Bangladesh**, pays se trouvant dans notre top 4 sur la production de textile pour notre marque, qui réalise le plus de hausse en valeur ajoutée dans son industrie manufacturière en 10 ans (**+ 313.9%)**. Malheureusement la Chine, notre 2e pays de fabrication, ne fournit pas de données sur cet indicateur. Nous remarquons la tendance inverse pour le Brésil (**- 50.1%** en 10 ans). Les autres pays présentent des hausses plus légères mais qui ne sont toutefois pas négligeables puisqu'il s'agit de billions de $.</br>
**Concernant la valeur ajoutée entre les pays** : nous constatons que l'Inde surpasse les autres pays en terme de volume. Par exemple elle réalise **5x plus de valeur ajoutée que le Bangladesh en 2020**. Si le Bangladesh est caractérisé par un fort taux d'évolution, l'Inde surpasse les chiffres en terme de valeur absolue.</br></br>
Qu'en est-il à présent de la répartition des valeurs ajoutées selon le secteur manufacturier pour nos pays d'intérêt ?

In [ ]:
# stacked bar des secteurs manufacturiers
y1 = [15.4, 11.8, math.nan, math.nan, 15.5, 10.8, 12.4, math.nan, 16.3, 19] # liste qui contient les valeurs brutes pour obtenir un affichage cohérent
y2 = [56.8, 10, math.nan, math.nan, 14.4, 16.1, 16.2, math.nan, 2.9, 5.8]
y3 = [3.6, 24.5, math.nan, math.nan, 18.3, 21.7, 30.3, math.nan, 22.6, 17]
y4 = [3.1, 10.8, math.nan, math.nan, 5.9, 9.3, 4.2, math.nan, math.nan, 14.7]
y5 = [21.1, 42.8, math.nan, math.nan, 45.9, 42.1, 37, math.nan, 58.1, 43.5]

trace_alimentation = go.Bar(name='Alimentation, boisson, tabac', x=df_countries_selected.index, y=df_countries_selected["Alimentation, boisson, tabac - 2020 (en %)"], text=y1, textposition='inside', marker_color='lightblue')
trace_textiles = go.Bar(name='Textiles, vêtements', x=df_countries_selected.index, y=df_countries_selected["Textiles, vêtements - 2020 (en %)"], text=y2, textposition='inside', marker_color='skyblue')
trace_machines = go.Bar(name='Machines, transport', x=df_countries_selected.index, y=df_countries_selected["Machines, mat. de transport - 2020 (en %)"], text=y3, textposition='inside', marker_color='steelblue')
trace_chimiques = go.Bar(name='Prod. chimiques', x=df_countries_selected.index, y=df_countries_selected["Prod. chimiques - 2020 (en %)"], text=y4, textposition='inside', marker_color='darkblue')
trace_autres = go.Bar(name='Autres', x=df_countries_selected.index, y=df_countries_selected["Autres - 2020 (en %)"], text=y5, textposition='inside', marker_color='black')

stacked_bar_manufacturing = go.Figure(data=[trace_alimentation, trace_textiles, trace_machines, trace_chimiques, trace_autres])
stacked_bar_manufacturing.update_layout(title="Valeur ajoutée par type d'industrie manufacturière en 2020", barmode='stack', plot_bgcolor="rgba(0,0,0,0)", yaxis=dict(showgrid=True, gridcolor='lightgray', range=[0, 100], tickformat='.%'), yaxis_title="% par rapport à la val. ajoutée totale")
stacked_bar_manufacturing.add_annotation(x=2.5, y=0.1, text="Médiane industrie textile : 3.6%", showarrow=False, font=dict(color='red'), yshift=10)
stacked_bar_manufacturing.show()

C'est à nouveau le **Bangladesh** qui présente le taux le plus important de valeur ajoutée quant à son industrie textile (**56.8%** de la val. ajoutée totale tous secteurs confondus). La Chine présente cette fois-ci des données mais l'industrie textile est la moins représentée, comparativement à la catégorie "autres" (**42.8%**). Malheureusement nous n'avons pas de données pour l'Inde et le Pakistan. Les autres pays sont surtout caractérisés par une importante représentation de la catégorie "autres". Il est intéressant de visualiser la différence entre la valeur du Bangladesh par rapport à la médiane mondiale (**+56.8% contre 3.6% !**)

### Zoom sur le travail des enfants et leur accès à l'éducation

Effectuons un rapide aperçu de nos données concernant le travail des enfants à l'aide de la fonction describe(). Nous précisons qu'à l'issue d'un travail exploratoire réalisé en amont, nous ne sélectionnons que les données qui, à notre sens, apportent une information pertinente à notre étude (par ex. nous excluons toutes les comparaisons liées au genre car elles n'indiquent pas de différences significatives, et il en sera de même pour tout notre rapport).

In [ ]:
df[["Total", "Work only", "Study and work", "Agriculture", "Manufacturing", "Services"]].describe().round(2)

,Total,Work only,Study and work,Agriculture,Manufacturing,Services
count,97.00,97.00,97.00,63.00,63.00,63.00
mean,19.70,24.04,75.96,67.44,6.06,24.07
std,16.48,22.67,22.67,21.08,6.27,18.05
min,1.20,0.00,10.70,14.40,0.00,0.80
25%,5.60,5.00,63.40,55.10,1.35,9.75
50%,13.30,18.20,81.80,69.80,3.30,22.20
75%,31.80,36.60,95.00,85.10,9.50,34.30
max,63.90,89.30,100.00,98.90,27.60,81.00


Un comparatif nous laisse entrevoir à nouveau l'écart entre certaines médianes et le max des distributions. En moyenne, dans le monde, **19.70%** des enfants travaillent. Parmi eux, **24%** ne font que travailler alors que **76%** d'entre eux travaillent et vont en parallèle à l'école. Le secteur le plus représenté est l'**agriculture**, puis les **services**, et enfin la **manufacture**. Les moyennes sont certainement influencées par les valeurs extrèmes, aussi, nous choisissons encore une fois de regarder préférentiellement les médianes. Les coefficients de variation (sd / moy * 100) sont tous très élevés, largement supérieurs à 50% ce qui signifie que les distributions sont très hétérogènes, ce qui semble normal puisque nous considérons dans un même ensemble tous les pays du monde. C'est un autre argument pour éviter d'utiliser la moyenne comme un référentiel.</br></br>
Nous allons nous pencher sur les pays de fabrication des vêtements Vertbaudet. Qu'en est-il de l'indicateur de travail des enfants ?

In [ ]:
# réalisons un barplot sur le travail des enfants
total_child_labor = go.Figure(data=go.Bar(x=df_countries_selected.index, y=df_countries_selected["Total"], text=df_countries_selected["Total"], textposition='outside', marker_color='lightblue'))
total_child_labor.update_layout(title="Proportion d'enfants âgés de 7 à 14 ans impliqués dans une activité économique au moins 1h par semaine", plot_bgcolor="rgba(0,0,0,0)", yaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis_title="% par rapport à la population des 7-14 ans")
total_child_labor.show()

Le pays qui comporte le plus d'enfants travailleurs est le **Pakistan**, puis arrivent le **Vietnam** et le **Bangladesh**. Il est à noter que tous les pays de fabrication des vêtements Vertbaudet, présentent un taux de travail infantile inférieur à la médiane mondiale qui est de **13.30%**.

In [ ]:
# cherchons quel est le pays du dataset qui emploie le plus d'enfants et quel est le %
country_max = df.index[df["Total"] == max(df["Total"])][0]
print(country_max, max(df["Total"]))

Guinea-Bissau 63.9


Dans quels secteurs d'activité travaillent ces enfants ?

In [ ]:
# visualisons cela sur un stacked bar
y1 = [39.3, math.nan, 56.5, 76.1, 48.5, 57.1, 77.1, 55.5, math.nan, 56.1]
y2 = [27.3, math.nan, 27.6, 6.7, 11.2, 14.3, 5.9, 17.9, math.nan, 8.4]
y3 = [33.3, math.nan, 10.4, 14.6, 33.3, 27.1, 16.2, 21.9, math.nan, 35.6]

trace_agriculture = go.Bar(name='Agriculture', x=df_countries_selected.index, y=df_countries_selected["Agriculture"], text=y1, textposition='inside', marker_color='lightblue')
trace_manufacture = go.Bar(name='Manufacture', x=df_countries_selected.index, y=df_countries_selected["Manufacturing"], text=y2, textposition='inside', marker_color='steelblue')
trace_services = go.Bar(name='Services', x=df_countries_selected.index, y=df_countries_selected["Services"], text=y3, textposition='inside', marker_color='darkblue')

stacked_bar_labor = go.Figure(data=[trace_agriculture, trace_manufacture, trace_services])
stacked_bar_labor.update_layout(title="Répartition des enfants travailleurs selon le secteur en 2020", barmode='stack', plot_bgcolor="rgba(0,0,0,0)", yaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis_title="% par rapport à la pop. d'enfants travailleurs")
stacked_bar_labor.show()

C'est dans l'**agriculture** que la majorité des enfants travaillent, quel que soit le pays, puis dans la **manufacture** et les **services**. C'est l'**Inde** et le **Bangladesh** qui présentent le plus fort taux de travail des enfants dans l'industrie manufacturière (**27.6% et 27.3%** respectivement). Notons un biais dans les données présentées qui n'atteignent pas toutes la barre des 100% en terme de proportions, cela étant probablement dû à un biais d'arrondissement des données brutes, à moins qu'une catégorie soit manquante dans le dataset de départ (une catégorie 'Autres'). Nous avons tout de même privilégié cette représentation qui nous semble la plus visuelle.

Voyons maintenant la répartition d'enfants selon leur activité : travail uniquement ou activité mixte travail/école.

In [ ]:
# réalisons un graphique en radar pour les répartitions d'activités
df_countries_selected_sorted_labor = df_countries_selected.sort_values(by=['Work only'], ascending=False)

morocco_index = df_countries_selected_sorted_labor.index.get_loc('Morocco') # renvoie l'index associé au pays dans le dataframe
pakistan_index = df_countries_selected_sorted_labor.index.get_loc('Pakistan')

labor_radar = go.Figure()
labor_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_labor["Work only"][i] for i in range(len(df_countries_selected_sorted_labor))], theta=df_countries_selected_sorted_labor.index, fill='toself', name='Travail unique', line_color='steelblue'))
labor_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_labor["Study and work"][i] for i in range(len(df_countries_selected_sorted_labor))], theta=df_countries_selected_sorted_labor.index, fill='toself', name='Travail et école', line_color='darkblue'))
labor_radar.update_layout(title="Répartition des enfants travailleurs selon une activité unique ou mixte (en %)", polar=dict(radialaxis=dict(visible=True, range=[0, 100])), plot_bgcolor="white")

# ajoutons des marqueurs pour le Maroc et le Pakistan
labor_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_labor["Work only"][morocco_index]], theta=[df_countries_selected_sorted_labor.index[morocco_index]], mode='markers', name='Maroc', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
labor_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_labor["Work only"][pakistan_index]], theta=[df_countries_selected_sorted_labor.index[pakistan_index]], mode='markers', name='Pakistan', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
labor_radar.show()

Ce graphique en 'radar' représente la proportion d'enfants, parmi l'ensemble des enfants travailleurs, qui travaillent uniquement, et ceux qui travaillent et vont à l'école. Les valeurs sont situées sur les axes radiaux (échelle de 0 à 100%). Parmi les enfants travailleurs, nous observons que dans des pays comme le **Bangladesh**, le **Pakistan**, l'**Inde** et le **Maroc**, la majorité des enfants qui travaillent ne vont pas à l'école (respectivement **58.1%, 87.4%, 82.5% et 84.5%**). Nous avons pointé le Maroc et le Pakistan qui présentent les plus forts taux de travail unique. La tendance diffère pour le **Portugal**, la **Turquie**, le **Vietnam** et le **Brésil** (**respectivement 96.4%, 61.2%, 81% et 96%** des enfants vont à l'école en parallèle dans ces pays).</br></br>
Nous pouvons conclure de ce tour d'horizon sur le travail des enfants, que les pays les plus producteurs de textile présentent encore actuellement des taux de travail infantile non négligeables, ce qui porte atteinte aux droits des enfants. Par ailleurs, la majorité des enfants ont le travail comme unique occupation et une minorité vont aussi à l'école.</br>
Comme nous l'avons évoqué dans notre introduction, il existe dans ces pays une causalité circulaire entre diverses variables concernant la pauvreté et le travail forcé, celui des enfants en faisant partie. Même si ces phénomènes sont multifactoriels, il est intéressant pour nous de relever ici une sorte de contingence.</br></br>
Nous avons évoqué à travers les sources mentionnées, que le travail infantile est le résultat de l'extrême pauvreté existant dans certains pays, obligeant les enfants à travailler pour apporter un revenu supplémentaire à leur famille. Observons les données sur la pauvreté pour comparer nos pays.

### Zoom sur la pauvreté

In [ ]:
# réalisons un graphique en radar sur la pauvreté
df_countries_selected_sorted_poverty = df_countries_selected.sort_values(by=["Pop. < 6.85$ / jour"], ascending=False)

india_index = df_countries_selected_sorted_poverty.index.get_loc('India')
pakistan_index = df_countries_selected_sorted_poverty.index.get_loc('Pakistan')
bangladesh_index = df_countries_selected_sorted_poverty.index.get_loc('Bangladesh')

poverty_radar = go.Figure()
poverty_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_poverty["Pop. < 2.15$ / jour"][i] for i in range(len(df_countries_selected_sorted_poverty))], theta=df_countries_selected_sorted_poverty.index, fill='toself', name='<2.15$/jr.', line_color='darkblue'))
poverty_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_poverty["Pop. < 3.65$ / jour"][i] for i in range(len(df_countries_selected_sorted_poverty))], theta=df_countries_selected_sorted_poverty.index, fill='toself', name='<3.65$/jr.', line_color='steelblue'))
poverty_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_poverty["Pop. < 6.85$ / jour"][i] for i in range(len(df_countries_selected_sorted_poverty))], theta=df_countries_selected_sorted_poverty.index, fill='toself', name='<6.85$/jr.', line_color='lightblue'))
poverty_radar.update_layout(title="Taux de pauvreté selon trois seuils (en % de la pop. totale)", polar=dict(radialaxis=dict(visible=True, range=[0, 100])), plot_bgcolor="white")
poverty_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_poverty["Pop. < 6.85$ / jour"][india_index]], theta=[df_countries_selected_sorted_poverty.index[india_index]], mode='markers', name='Inde', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
poverty_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_poverty["Pop. < 6.85$ / jour"][pakistan_index]], theta=[df_countries_selected_sorted_poverty.index[pakistan_index]], mode='markers', name='Pakistan', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
poverty_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_poverty["Pop. < 6.85$ / jour"][bangladesh_index]], theta=[df_countries_selected_sorted_poverty.index[bangladesh_index]], mode='markers', name='Bangladesh', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
poverty_radar.show()

Ce graphique en radar présente les taux de pauvreté selon différents seuils pour les pays sélectionnés. Les axes radiaux représentent les seuils de pauvreté, allant du seuil le plus bas (moins de 2.15 dollars par jour) au seuil le plus élevé (moins de 6.85 dollars par jour). Ce qui est intéressant et flagrant à observer sur ce radar, c'est que les 3 premiers pays producteurs de textile chez Vertbaudet (**Bangladesh, Inde, Pakistan**) sont encore une fois regroupés, ce sont eux qui concentrent le plus de populations pauvres.

### Zoom sur l'accès à l'eau potable

L'industrie du textile et en particulier la fast fashion consomme énormément d'eau ; nous souhaiterions savoir si les populations (donc, les enfants) vivant dans les pays de fabrication ont des difficultés d'accès à l'eau potable, comparativement à d'autres pays.

In [ ]:
# barplot sur l'accès à l'eau potable par rapport à l'eau prélevée
compare_water = go.Figure(data=[go.Bar(name='Eau disponible', x=df_countries_selected.index, y=df_countries_selected["Ressources internes en eau (b.m3)"], yaxis='y', offsetgroup=1, text=df_countries_selected["Ressources internes en eau (b.m3)"], textposition='outside', marker_color='lightblue'),
                                go.Bar(name='Eau prélevée', x=df_countries_selected.index, y=df_countries_selected["Prélèvements annuels (b.m3)"], yaxis='y', offsetgroup=2, text=df_countries_selected["Prélèvements annuels (b.m3)"], textposition='outside', marker_color='steelblue')])
compare_water.update_layout(title="Comparatif de l'eau disponible vs. prélevée annuellement pour les pays de fabrication", barmode='group', plot_bgcolor="rgba(0,0,0,0)", yaxis=dict(showgrid=True, gridcolor='lightgray', range=[0, 3000]), yaxis_title="Vol. d'eau disponible vs. prélevée par an (en b. m3)",
    shapes=[dict(type="line", yref="y", y0=26.9, y1=26.9, xref="paper", x0=0, x1=1, line=dict(color="red")), dict(type="line", yref="y", y0=2.2, y1=2.2, xref="paper", x0=0, x1=1, line=dict(color="red"))])
compare_water.add_annotation(x=-1, y=90, text="26.9 disp", showarrow=False, font=dict(color='red'), yshift=10)
compare_water.add_annotation(x=-1, y=2.2, text="2.2 prel", showarrow=False, font=dict(color='red'), yshift=10)
compare_water.add_annotation(x='Bangladesh', y=400, text="Par habitant : 634 m3", showarrow=False, font=dict(color='white', size=12), align='center', bgcolor='darkblue', opacity=0.7)
compare_water.add_annotation(x='Pakistan', y=400, text="Par habitant : 246 m3", showarrow=False, font=dict(color='white', size=12), align='center', bgcolor='darkblue', opacity=0.7)
compare_water.add_annotation(x='Morocco', y=400, text="Par habitant : 799 m3", showarrow=False, font=dict(color='white', size=12), align='center', bgcolor='darkblue', opacity=0.7)
compare_water.show()

Ce graphique présente les quantités d'eau disponible dans les ressources du pays, c'est-à-dire issues des fleuves et nappes phréatiques, comparativement aux eaux prélevées dans les bassins de stockage, en billions de mètres cubes. Nous y avons également fait figurer les ressources en eau par habitant pour les 4 pays les plus producteurs. Il est intéressant de constater les volumes considérables d'eau prélevée pour la **Chine**, l'**Inde** et le **Pakistan**. Ce dernier d'ailleurs prélève plus de 3x le volume d'eau disponible, ce qui est interpelant concernant l'accès à l'eau pour les populations (car c'est aussi le pays qui comporte le moins de ressources en eau par habitant : 246 m3 seulement). En effet, nous avons vu dans la partie précédente, l'ampleur de l'empreinte eau dans les pays les plus producteurs, et il est inquiétant de se projeter si l'industrie continue sur ce modèle, sachant que l'eau deviendra de plus en plus rare.

Voyons à présent pour quel usage l'eau est prélevée.

In [ ]:
# visualisons sur un barplot
compare_withdrawals = go.Figure(data=[go.Bar(name='Agriculture', x=df_countries_selected.index, y=df_countries_selected["Prélèvements agriculture (en %)"], yaxis='y', offsetgroup=1, text=df_countries_selected["Prélèvements agriculture (en %)"], marker_color='lightblue'),
                                      go.Bar(name='Industrie', x=df_countries_selected.index, y=df_countries_selected["Prélèvements industrie (en %)"], yaxis='y', offsetgroup=2, text=df_countries_selected["Prélèvements industrie (en %)"], marker_color='steelblue'),
                                      go.Bar(name='Domestique', x=df_countries_selected.index, y=df_countries_selected["Prélèvements domestiques (en %)"], yaxis='y', offsetgroup=3, text=df_countries_selected["Prélèvements domestiques (en %)"], marker_color='darkblue')])
compare_withdrawals.update_layout(title="Usage des prélèvements en eau", barmode='group', plot_bgcolor="rgba(0,0,0,0)", yaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis_title="% d'eau prélevée par rapport au total")
compare_withdrawals.show()

Nous observons que c'est en grande majorité l'agriculture, et ce dans tous les pays sélectionnés, qui pompe l'eau disponible en grandes quantités.

### Zoom sur l'accès à l'énergie

In [ ]:
# visualisons sur un barplot
compare_electricity = go.Figure(data=[go.Bar(name='Cuisine', x=df_countries_selected.index, y=df_countries_selected["Accès aux énergies propres pour cuisiner - 2020 (en %)"], yaxis='y', offsetgroup=1, text=df_countries_selected["Accès aux énergies propres pour cuisiner - 2020 (en %)"], marker_color='lightblue'),
                                      go.Bar(name='Energ. renouv.', x=df_countries_selected.index, y=df_countries_selected["Consommation d'énergie renouvelable - 2020 (en %)"], yaxis='y', offsetgroup=2, text=df_countries_selected["Consommation d'énergie renouvelable - 2020 (en %)"], marker_color='steelblue')])
compare_electricity.update_layout(title="Accès aux énergies propres pour cuisiner et aux énergies renouvelables", barmode='group', plot_bgcolor="rgba(0,0,0,0)", yaxis=dict(showgrid=True, gridcolor='lightgray'), yaxis_title="% de la pop. totale")
compare_electricity.show()

### Zoom sur la malnutrition

Intéressons-nous à la malnutrition et au retard de croissance, qui selon nous sont des conséquences de la pauvreté. Nous ne retenons pas le surpoids que nous considérons comme un trouble qui revêt une dimension différente et n'est peut-être pas la problématique la plus préoccupante pour les populations qui nous intéressent.

In [ ]:
# réalisons un radar pour la prévalence de la malnutrition et du retard de croissance
df_countries_selected_sorted_nutrition = df_countries_selected.sort_values(by=["Prévalence du retard de croissance - 2020 (en %)"], ascending=False)

bangladesh_index = df_countries_selected_sorted_nutrition.index.get_loc('Bangladesh')
india_index = df_countries_selected_sorted_nutrition.index.get_loc('India')
pakistan_index = df_countries_selected_sorted_nutrition.index.get_loc('Pakistan')

nutrition_radar = go.Figure()
nutrition_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_nutrition["Prévalence de la malnutrition - 2020 (en %)"][i] for i in range(len(df_countries_selected_sorted_nutrition))], theta=df_countries_selected_sorted_nutrition.index, fill='toself', name='Malnutrition', line_color='darkblue'))
nutrition_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_nutrition["Prévalence du retard de croissance - 2020 (en %)"][i] for i in range(len(df_countries_selected_sorted_nutrition))], theta=df_countries_selected_sorted_nutrition.index, fill='toself', name='Retard croiss.', line_color='lightblue'))
nutrition_radar.update_layout(title="Prévalences de la malnutrition et du retard de croissance (en %)", polar=dict(radialaxis=dict(visible=True, range=[0, 40])), plot_bgcolor="white")
nutrition_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_nutrition["Prévalence du retard de croissance - 2020 (en %)"][bangladesh_index]], theta=[df_countries_selected_sorted_nutrition.index[bangladesh_index]], mode='markers', name='Bangladesh', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
nutrition_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_nutrition["Prévalence du retard de croissance - 2020 (en %)"][india_index]], theta=[df_countries_selected_sorted_nutrition.index[india_index]], mode='markers', name='India', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
nutrition_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_nutrition["Prévalence du retard de croissance - 2020 (en %)"][pakistan_index]], theta=[df_countries_selected_sorted_nutrition.index[pakistan_index]], mode='markers', name='Pakistan', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
nutrition_radar.show()

Ce radar pointe à nouveau le **Bangladesh**, l'**Inde** et le **Pakistan** comme dominants concernant la proportion d'enfants malnutris (respectivement **30.2%, 30.9% et 36.7%**), ou présentant un retard de croissance (respectivement **11.4%, 16.3% et 16.9%**).

### Zoom sur la mortalité

In [ ]:
# barplot de comparaison des espérances de vie avec le référentiel de la France
compare_life_expectancy = go.Figure(data=[go.Bar(name='2020', y=df_countries_selected.index, x=df_countries_selected["Espérance de vie - 2020"], xaxis='x', offsetgroup=1, text=df_countries_selected["Espérance de vie - 2020"], marker_color='pink', orientation='h')])
compare_life_expectancy.update_layout(title="Espérance de vie en 2020 pour chaque pays", barmode='group', plot_bgcolor="rgba(0,0,0,0)", xaxis_title="Espérance de vie (en années)", yaxis_categoryorder='total descending', yaxis_title="Pays", width=1300, shapes=[dict(type="line", yref="paper", y0=0, y1=1, xref="x", x0=82.25, x1=82.25, line=dict(color="red"))])
compare_life_expectancy.add_annotation(x=77, y=10, text="France : 82.25 ans", showarrow=False, font=dict(color='red'), yshift=10)
compare_life_expectancy.show()

Nous voyons à nouveau que les pays les plus reculés de la France en terme d'espérance de vie sont : **Pakistan** (**67 ans**), **Inde** (**70 ans**), **Bangladesh** (**73 ans**). Soir respectivement **15.25, 12.25 et 9.25** points de moins que la France.

In [ ]:
# scatterpolar pour les taux de mortalité selon les âges des enfants
df_countries_selected_sorted_mortality = df_countries_selected.sort_values(by=['Taux de mortalité < 5 ans - 2020 (en ‰)'], ascending=False)

bangladesh_index = df_countries_selected_sorted_mortality.index.get_loc('Bangladesh') # renvoie la position de l'index correspondant
india_index = df_countries_selected_sorted_mortality.index.get_loc('India')
pakistan_index = df_countries_selected_sorted_mortality.index.get_loc('Pakistan')

mortality_radar = go.Figure()
mortality_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_mortality["Taux de mortalité néonatale - 2020 (en ‰)"][i] for i in range(len(df_countries_selected_sorted_mortality))], theta=df_countries_selected_sorted_mortality.index, fill='toself', name='Néonatale', line_color='darkblue'))
mortality_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_mortality["Taux de mortalité infantile - 2020 (en ‰)"][i] for i in range(len(df_countries_selected_sorted_mortality))], theta=df_countries_selected_sorted_mortality.index, fill='toself', name='Infantile', line_color='steelblue'))
mortality_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_mortality["Taux de mortalité < 5 ans - 2020 (en ‰)"][i] for i in range(len(df_countries_selected_sorted_mortality))], theta=df_countries_selected_sorted_mortality.index, fill='toself', name='< 5 ans', line_color='lightblue'))
mortality_radar.update_layout(title="Taux de mortalité selon l'âge de l'enfant (pour 1000 naissances en un an)", polar=dict(radialaxis=dict(visible=True, range=[0, 70])), plot_bgcolor="white")
mortality_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_mortality["Taux de mortalité < 5 ans - 2020 (en ‰)"][bangladesh_index]], theta=[df_countries_selected_sorted_mortality.index[bangladesh_index]], mode='markers', name='Bangladesh', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
mortality_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_mortality["Taux de mortalité < 5 ans - 2020 (en ‰)"][india_index]], theta=[df_countries_selected_sorted_mortality.index[india_index]], mode='markers', name='Inde', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
mortality_radar.add_trace(go.Scatterpolar(r=[df_countries_selected_sorted_mortality["Taux de mortalité < 5 ans - 2020 (en ‰)"][pakistan_index]], theta=[df_countries_selected_sorted_mortality.index[pakistan_index]], mode='markers', name='Pakistan', marker=dict(color='red', size=10, line=dict(color='black', width=2))))
mortality_radar.show()

Nous observons que les plus hauts taux de mortalité, à tous les âges des enfants, se situent au **Bangladesh**, **Inde**, **Pakistan** : pour la mortalité néonatale on mesure respectivement **18‰, 20‰ et 40‰** ; pour la mortalité infantile on note respectivement **24‰, 27‰ et 54‰** ; pour la mortalité < 5 ans, on relève respectivement **29‰, 33‰ et 65‰**.

**Dans cette première partie sur les liens entre la fast fashion et les données sur les populations de la Banque Mondiale, nous avons mis en évidence à mesure de notre progression, un pattern de résultats similaire concernant les pays figurant dans notre top 3 (si l'on élimine la Chine qui est plutôt un pays riche) des pays les plus gros producteurs de vêtements pour Vertbaudet, à savoir le Bangladesh, l'Inde et le Pakistan. Nous avons vu dans un premier temps l'ampleur de leur empreinte carbone et eau, et dans un deuxième temps, que ce sont ces pays qui, alors qu'ils génèrent de la richesse (forte valeur ajoutée) de par leur industrie, présentent aussi les plus hauts taux de travail infantile unique sans scolarisation, pauvreté, malnutrition, mortalité infantile, et les plus bas taux d'accès aux énergies pour cuisiner. Nous avons vu la quantité d'eau prélevée par ces pays, alors que les ressources en eau par habitant y sont moindres que dans les autres pays. Notre constat ne souligne pas, bien entendu, de relation de cause à effet, mais simplement une contigence importante à relever. Notre interprétation met en évidence des inégalités dans ces pays producteurs, entre l'importance accordée à la production pour générer richesse et profit, au détriment des populations qui y vivent. Outre cette inégalité, nos calculs initiaux sur l'empreinte carbone et eau de ces pays nous montre à quel point cette industrie est néfaste pour l'environnement dans lequel vivent les enfants qui y habitent, portant atteinte à leur droit à évoluer dans un milieu sain. Nous allons à présent, dans notre seconde partie, nous focaliser sur la composition de ces vêtements et étudier le lien avec la santé des enfants qui les portent.**

# **Axe 2. La fast fashion et la santé des enfants consommateurs**

Nous avons pointé dans la partie précédente les pays de fabrication des vêtements Vertbaudet, et mis en évidence les tendances communes des 4 pays les plus producteurs, concernant leurs indicateurs environnementaux et sociaux négatifs.</br>
Nous souhaitons à présent nous pencher sur les enfants consommateurs (indirects, qui portent les vêtements achetés par leurs parents). 

Pour étudier cette question, nous nous pencherons sur la composition des vêtements du site Vertbaudet pour en extraire les types de matières (coton, laine, polyester, élasthanne, etc.) et en déterminer des taux, à partir des poids que nous aurons définis pour chaque vêtement. **Puis, nous définirons un score de toxicité pour chaque matière que nous assignerons à chaque vêtement, selon des critères que nous expliciterons**. 

## Scraping des vêtements et leurs composés sur le site Vertbaudet et calcul d'un score de toxicité

Nous allons récupérer l'ensemble des articles disponibles sur les pages bébés, filles et garçons du site, les stocker dans des dataframes et concaténer les trois dataframes obtenus en un unique. Puis nous nettoierons les données de manière à obtenir un résultat exploitable, et nous définirons un **score de toxicité** pour chaque matière. Méthodologiquement, il n'a pas été possible de récupérer des chiffres précis et tangibles mesurant la présence d'agents toxiques dans chaque matière. Les articles mentionnant ces idées présentent les matières synthétiques, comme nous l'avons vu, comme les plus néfastes, sans toutefois donner de chiffres, ou sans différencier les matières entre elles, ce qui n'est pas suffisamment précis pour nous offrir un référentiel calculatoire. Les études portant sur la présence d'agents chimiques et leurs effets sur la santé portent toutes sur des tests en laboratoire à partir d'échantillons de vêtements, ce qui ne nous permet pas d'extrapoler sur les collections présentes chez Vertbaudet. Aussi, nous allons nous baser sur des ressources afin de nous forger des critères pour placer chaque matière sur une échelle allant de **0 (pas du tout toxique)** à **5 (extrêmement toxique)**. Nous les expliciterons plus loin dans notre code.

In [ ]:
# ----- pages bébés -----

# récupérons les titres et les URL de chaque article pour les 27 pages
pages_bebes = [str(i) for i in range(1,28)]
all_titres = []
start_time = time()
page_count = 0
req = 0

for page in pages_bebes:
  url = 'https://www.vertbaudet.fr/bebe/page'+ page +'.htm'
  response = requests.get(url, allow_redirects=False)
  sleep(randint(8,15))
  req += 1
  elapsed_time = time() - start_time
  print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
  clear_output(wait = True)
  if response.status_code == 200:
    page_count += 1
  soup = BeautifulSoup(response.text, 'html.parser')
  body = soup.body
  container = body.find_all('a', class_='omnitureproductlink product-image-link')
  all_titres = all_titres + container

Request: 27; Frequency: 0.08054661753530698 requests/s


In [ ]:
# récupérons les compositions pour chaque article bébé pour les 27 pages
liens = [l["href"] for l in all_titres] # pour avoir les contenus de la balise href
pages_liens = [str(i) for i in liens] # pour automatiser les URL dans la boucle
page_line_count = len(pages_liens) # pour baliser le nb de requêtes
composants = []
composes = []
composants_final = []
start_time = time()
req = 0

for page in pages_liens:
  url = 'https://www.vertbaudet.fr'+ page
  response = requests.get(url)
  sleep(1)
  req += 1
  elapsed_time = time() - start_time
  print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
  clear_output(wait = True)
  if response.status_code != 200:
    warn('Request: {}; Status code: {}'.format(req, response.status_code))
  if req > page_line_count:
    warn('Nombre de requêtes trop important')
    break
  soup = BeautifulSoup(response.text, 'html.parser')
  body = soup.body
  container = body.find_all('p', itemprop='description')
  composants = composants + container
  composants_final = [t.text.replace('\xa0', ' ') for t in composants]

  for text in composants_final:
    matches = re.findall(r'\d+% \w+', text)
    if len(matches) == 0:
      matches = re.findall(r'pur \w+', text)
  if len(matches) != 0:
    composes.insert(req,matches)
  if len(composes) < req:
    composes.insert(req,'[]')

Request: 1555; Frequency: 0.5391694849351871 requests/s


In [ ]:
# récupérons le dataframe
df_bebes = pd.DataFrame({
    'titres': all_titres,
    'liens': liens,
    'composes': composes
})
df_bebes.head(2)

,titres,liens,composes
0,[Sweat bébé encolure en broderie anglaise - ro...,/sweat-bebe-encolure-en-broderie-anglaise-rose...,"[98% coton, 2% élasthanne]"
1,[Ensemble combinaison + bandeau imprimés bébé ...,/ensemble-combinaison-bandeau-imprimes-bebe-fi...,[100% cotonCouleur]


In [ ]:
len(df_bebes) # index 0 à 1555

1556

In [ ]:
# ----- pages filles -----

# récupérons les titres et les URL de chaque article pour les 33 pages
pages_filles = [str(i) for i in range(1,34)]
all_titres = []
start_time = time()
page_count = 0
req = 0

for page in pages_filles:
  url = 'https://www.vertbaudet.fr/fille/page'+ page +'.htm'
  response = requests.get(url, allow_redirects=False)
  sleep(randint(8,15))
  req += 1
  elapsed_time = time() - start_time
  print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
  clear_output(wait = True)
  if response.status_code == 200:
    page_count += 1
  soup = BeautifulSoup(response.text, 'html.parser')
  body = soup.body
  container = body.find_all('a', class_='omnitureproductlink product-image-link')
  all_titres = all_titres + container

Request: 33; Frequency: 0.08292700839107571 requests/s


In [ ]:
# récupérons les compositions pour chaque article fille pour les 33 pages
liens = [l["href"] for l in all_titres] # pour avoir les contenus de la balise href
pages_liens = [str(i) for i in liens] # pour automatiser les URL dans la boucle
page_line_count = len(pages_liens) # pour baliser le nb de requêtes
composants = []
composes = []
composants_final = []
start_time = time()
req = 0

for page in pages_liens:
  url = 'https://www.vertbaudet.fr'+ page
  response = requests.get(url)
  sleep(1)
  req += 1
  elapsed_time = time() - start_time
  print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
  clear_output(wait = True)
  if response.status_code != 200:
    warn('Request: {}; Status code: {}'.format(req, response.status_code))
  if req > page_line_count:
    warn('Nombre de requêtes trop important')
    break
  soup = BeautifulSoup(response.text, 'html.parser')
  body = soup.body
  container = body.find_all('p', itemprop='description')
  composants = composants + container
  composants_final = [t.text.replace('\xa0', ' ') for t in composants]

  for text in composants_final:
    matches = re.findall(r'\d+% \w+', text)
    if len(matches) == 0:
      matches = re.findall(r'pur \w+', text)
  if len(matches) != 0:
    composes.insert(req,matches)
  if len(composes) < req:
    composes.insert(req,'[]')

Request: 1833; Frequency: 0.5167835104906204 requests/s


In [ ]:
# récupérons le dataframe
df_filles = pd.DataFrame({
    'titres': all_titres,
    'liens': liens,
    'composes': composes
})
df_filles.head(2)

,titres,liens,composes
0,[T-shirt fille animation frisou et détails iri...,/t-shirt-fille-animation-frisou-et-details-iri...,[pur coton]
1,[T-shirt à motif irisé fille manches courtes v...,/t-shirt-a-motif-irise-fille-manches-courtes-v...,[100% coton]


In [ ]:
len(df_filles) # index 0 à 1832

1833

In [ ]:
# ----- pages garçons -----

# récupérons les titres et les URL de chaque article pour les 24 pages
pages_garcons = [str(i) for i in range(1,25)]
all_titres = []
start_time = time()
page_count = 0
req = 0

for page in pages_garcons:
  url = 'https://www.vertbaudet.fr/garcon/page'+ page +'.htm'
  response = requests.get(url, allow_redirects=False)
  sleep(randint(8,15))
  req += 1
  elapsed_time = time() - start_time
  print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
  clear_output(wait = True)
  if response.status_code == 200:
    page_count += 1
  soup = BeautifulSoup(response.text, 'html.parser')
  body = soup.body
  container = body.find_all('a', class_='omnitureproductlink product-image-link')
  all_titres = all_titres + container

Request: 24; Frequency: 0.08381125882569496 requests/s


In [ ]:
# récupérons les contenus de la balise href 
liens = [l["href"] for l in all_titres]

In [ ]:
# récupérons les compositions pour chaque article garçon pour les 24 pages
liens = [l["href"] for l in all_titres] # pour avoir les contenus de la balise href
pages_liens = [str(i) for i in liens] # pour automatiser les URL dans la boucle
page_line_count = len(pages_liens) # pour baliser le nb de requêtes
composants = []
composes = []
composants_final = []
start_time = time()
req = 0

for page in pages_liens:
  url = 'https://www.vertbaudet.fr'+ page
  response = requests.get(url)
  sleep(1)
  req += 1
  elapsed_time = time() - start_time
  print('Request: {}; Frequency: {} requests/s'.format(req, req/elapsed_time))
  clear_output(wait = True)
  if response.status_code != 200:
    warn('Request: {}; Status code: {}'.format(req, response.status_code))
  if req > page_line_count:
    warn('Nombre de requêtes trop important')
    break
  soup = BeautifulSoup(response.text, 'html.parser')
  body = soup.body
  container = body.find_all('p', itemprop='description')
  composants = composants + container
  composants_final = [t.text.replace('\xa0', ' ') for t in composants]

  for text in composants_final:
    matches = re.findall(r'\d+% \w+', text)
    if len(matches) == 0:
      matches = re.findall(r'pur \w+', text)
  if len(matches) != 0:
    composes.insert(req,matches)
  if len(composes) < req:
    composes.insert(req,'[]')

Request: 1289; Frequency: 0.5088750506883716 requests/s


In [ ]:
# récupérons le dataframe
df_garcons = pd.DataFrame({
    'titres': all_titres,
    'liens': liens,
    'composes': composes
})
df_garcons.head(2)

,titres,liens,composes
0,[T-shirt motif dinosaure géant garçon - menthe],/t-shirt-motif-dinosaure-geant-garcon-menthe.h...,[pur cotonCouleur]
1,[Tee-shirt animal ludique garçon - terracotta],/tee-shirt-animal-ludique-garcon-terracotta.ht...,[100% coton]


In [ ]:
len(df_garcons) # index 0 à 1288

1289

In [ ]:
# concaténons les 3 dataframes et exportons les
df_composes = pd.concat([df_bebes, df_filles, df_garcons], axis=0)
df_composes.to_csv('df_composes.csv')
files.download('df_composes.csv')

In [ ]:
uploaded = files.upload()
path = "df_composes.csv"
df_composes = pd.read_csv(path, usecols=['liens', 'composes'])

In [ ]:
# précisons que pour cette partie, nous avons fait de nombreux allers-retours entre le dataframe de depart et le dataframe d'arrivée en fonction des résultats observés
# ainsi, le nettoyage s'est fait en fonction des données visualisées et finalement inutiles ou non signifiantes

In [ ]:
# ajoutons une colonne pour différencier les collections bébé, fille et garçon
df_composes.loc[:1555, 'collection'] = 'bébé'
df_composes.loc[1556:3388, 'collection'] = 'fille'
df_composes.loc[3389:, 'collection'] = 'garçon'

In [ ]:
# nettoyons les liens pour obtenir uniquement le nom du vêtement et stockons le résultat dans une liste
liste_articles = df_composes['liens'].apply(lambda x: x.split('-')[0].replace('/',''))

In [ ]:
# remplaçons certains noms de vêtements pour limiter les value_counts
for i in range(len(liste_articles)):
  if liste_articles[i] == "t":
    liste_articles[i] = "tshirt"
  if liste_articles[i] == "tee":
    liste_articles[i] = "tshirt"
  if liste_articles[i] == "cire":
    liste_articles[i] = "ciret"
  if (liste_articles[i] == "dors") or (liste_articles[i] == "pyjashort") or (liste_articles[i] == "pyjacourt") or (liste_articles[i] == "bodyjama"):
    liste_articles[i] = "pyjama"
  if (liste_articles[i] == "chino") or (liste_articles[i] == "slim") or (liste_articles[i] == "pantacourt")  or (liste_articles[i] == "corsaire"):
    liste_articles[i] = "pantalon"
  if (liste_articles[i] == "shorty") or (liste_articles[i] == "bloomer"):  
    liste_articles[i] = "short"
  if liste_articles[i] == "beanie":
    liste_articles[i] = "bonnet"
  if liste_articles[i] == "ensemble":
    liste_articles[i] = "lot"
  if (liste_articles[i] == "guimpe") or (liste_articles[i] == "borsalino"):  
    liste_articles[i] = "chapeau"
  if (liste_articles[i] == "tregging"):  
    liste_articles[i] = "pantalon"
  if (liste_articles[i] == "combi") or (liste_articles[i] == "combishort") or (liste_articles[i] == "combipilote"):  
    liste_articles[i] = "combinaison"
  if (liste_articles[i] == "grande"):  
    liste_articles[i] = "robe"

In [ ]:
# créons une colonne articles avec la liste crée et supprimons la colonne liens
df_composes = df_composes.assign(articles=liste_articles)
df_composes.drop('liens', axis=1, inplace=True)

In [ ]:
# gardons uniquement les vêtements
to_drop = ['les', 'gourde', 'gourdes', 'coffret', 'cartable', 'montre', 'coupe', 'breloques', 'little', 'baby', 'sac', 'bouteille', 'lunettes', 'boite', 'le', 'trousse', 'pochette', 
           'parapluie', 'ceinture', 'tampon', 'bretelles', 'pack', 'couronne', 'serre', 'set', 'sous', 'kit', '3', 'cache']
df_composes = df_composes[~df_composes['articles'].isin(to_drop)]

In [ ]:
# remplaçons certaines chaînes orthographiées différemment pour harmoniser la lecture par Python
# mettons dans une catégorie les matières identiques pour éviter les quantités infimes 
# (ex. nylon = polyamide ; tencel, lyocell, modal, fibrane, rayonne = viscose ; alpaga, mohair, cachemire = laine)
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('pur','100%'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('cotonCouleur','coton'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('viscoseCouleur','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('élasthanneCouleur','élasthanne'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('laineCouleur','laine'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyesterCouleur','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('cotonBlouson','coton'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('économiques','coton'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('élasthanneEn','élasthanne'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyesterGarnissage','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('Coton','coton'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('Polyamide','polyamide'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('Lyocell','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('Polyester','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('Elasthanne','élasthanne'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('cotono','coton'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('acryliqueCouleur','acrylique'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('cachemireo','cachemire'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('viscoseGilet','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyesterIntérieur','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyesterDoublure','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyesterOuatinage','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyamideo','polyamide'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('lambswool','laine'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyamideDoublure','polyamide'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('acryliqueo','acrylique'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('laineo','laine'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('fibres','fibre'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('cpolyester','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('Nylon','nylon'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyamideCouleur','polyamide'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('élasthanneo','élasthanne'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('elastane','élasthanne'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('elasthane','élasthanne'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('elasthanne','élasthanne'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('élastodienne','élasthanne'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyestero','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('poyester','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('alpagao','alpaga'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('Tritan','tritan'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('fils','fibre'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('polyesterselon','polyester'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('Tencel','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('tencel','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('nylon','polyamide'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('modal','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('fibrane','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('rayonne','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('lyocell','viscose'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('modacrylique','acrylique'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('alpaga','laine'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('mohair','laine'))
df_composes['composes'] = df_composes['composes'].apply(lambda x: x.replace('cachemire','laine'))

In [ ]:
# supprimons les lignes avec des données aberrantes ou qui ne sont pas des matières textiles
df_composes['composes'] = df_composes['composes'].drop([325, 392, 642, 645, 692, 704, 710, 732, 867, 897, 938, 961, 965, 972, 1022, 1070, 1090, 1124, 1126, 1224, 1296, 2058, 2116, 2135, 2197, 
                                                        2212, 2279, 2345, 2374, 2391, 2469, 2503, 2541, 2552, 2591, 2645, 2764, 2807, 2812, 2817, 2831, 2868, 2920, 2955, 2963, 3045, 3048, 3108, 
                                                        3110, 3171, 3184, 3352, 3716, 3729, 3791, 3896, 3912, 3913, 3962, 4016, 4017, 4056, 4063, 4077, 4086, 4245, 4280, 4294, 4347, 4413])

In [ ]:
# supprimons les lignes où il y a des NaN à la place des composés
df_composes = df_composes.dropna(subset=['composes'])

In [ ]:
# transformons le type de données de la colonne 'composés' : nous avons des string et nous voulons des listes de string
df_composes['composes'] = df_composes['composes'].apply(ast.literal_eval)

In [ ]:
# effectuons une première visualisation de nos matières textiles en effectuant un value_counts
# dans un 1er temps, supprimons les pourcentages
def remove_pct(text):
    return [x.replace('%', '').replace('0','').replace('1','').replace('2','').replace('3','').replace('4','').replace('5','').replace('6','').replace('7','').replace('8','').replace('9','').replace(' ','').replace('.','') for x in text]

# appliquons la fonction à chaque élément de la colonne
compositions = df_composes['composes'].apply(remove_pct)
compositions = compositions.to_frame()

# empilons chaque élément et comptons-les puis transformons-les en %
exploded_df = compositions.explode('composes')
counts_pct = (exploded_df['composes'].value_counts(normalize=True).round(2))*100

In [ ]:
# affichons un graphique en barres pour avoir un ordre de grandeur des matières qui apparaissent dans le dataset (nous éliminons les matières < 1%)
sorted_data = sorted(counts_pct.items(), key=lambda x: x[1], reverse=False) # tri préalable des données
matieres = [data[0] for data in sorted_data] # listes pour stocker les matières et quantités triées
quantites = [data[1] for data in sorted_data]

fig_qt_material = make_subplots(rows=1, cols=1)
fig_qt_material.add_trace(go.Bar(x=[q for i, q in enumerate(quantites) if q >= 1], y=[m for i, m in enumerate(matieres) if quantites[i] >= 1], text=[q for i, q in enumerate(quantites) if q >= 1], textposition='outside', orientation='h', marker_color='lightblue'))
fig_qt_material.update_layout(title='Répartition des différentes matières textiles en %', xaxis_title='% par rapport au total', yaxis_title='Matières textiles', plot_bgcolor="rgba(0,0,0,0)", margin=dict(l=150, r=50, t=50, b=50))
fig_qt_material.show()

In [ ]:
# modifions notre dataframe avec chaque matière en colonne pour pouvoir les manipuler
matieres_premieres = counts_pct.index.tolist() # création d'une liste avec les matières

for matiere in matieres_premieres:
    df_composes[matiere] = pd.Series() # ajouter une colonne vide par matière au dataframe

In [ ]:
# remplissons les colonnes correspondantes avec les pourcentages en float
for index, row in df_composes.iterrows(): # itération sur chaque ligne du dataframe
    for composition in row['composes']: # itération sur la colonne composes
        for matiere in matieres_premieres: # itération sur la liste matieres_premieres
            if composition.find(matiere) != -1: # si la matière est trouvée dans la colonne composes (si l'index renvoyé est différent de -1)
                df_composes.loc[index, matiere] = float(composition.split('%')[0])/100 # extraire le chiffre du %, le convertir en float et l'assigner à la colonne appropriée

In [ ]:
# supprimons les colonnes non classables en tant que matière textile
df_composes = df_composes.fillna(0).drop('', axis=1)
df_composes = df_composes.fillna(0).drop('autres', axis=1) 
df_composes = df_composes.fillna(0).drop('fibre', axis=1) 
df_composes = df_composes.fillna(0).drop('gaze', axis=1) 
df_composes = df_composes.fillna(0).drop('polyuréthane', axis=1) 
df_composes = df_composes.fillna(0).drop('polyesterCouleur', axis=1) 
df_composes = df_composes.fillna(0).drop('cotonCouleur', axis=1) 
df_composes = df_composes.fillna(0).drop('linCouleur', axis=1) 

In [ ]:
# par défaut lorsqu'il y a plusieurs occurrences du même composé, Python ne retient que le 1er
# mais s'il y a 3 fois 100% composés différents, il met 3 fois 100% dans chaque colonne
# voici comment nous avons traité cela : 

In [ ]:
# fonction qui renvoie si le vêtement est unique ou un lot
def check_quantity(row):
  sum_composes = row[3:21].sum()
  if sum_composes <= 1:
    return 'unique'
  else:
    return 'lot'

df_composes['quantite'] = df_composes.apply(check_quantity, axis=1)

In [ ]:
# ajoutons une colonne 'poids' des vêtements à partir d'une grille récupérée sur ce site : https://rocketmf.com/en/weight#tab-5
def get_weight(row):
  article = row['articles']
  if article in ['tshirt', 'maillot', 'débardeur', 'brassiere']:
    return 0.065
  elif article in ['pantalon', 'jogging', 'sarouel']:
    return 0.185
  elif article in ['sweat', 'sweatshirt', 'chemise', 'polo', 'surchemise', 'ciret', 'pyjama', 'combinaison', 'surpyjama']:
    return 0.15
  elif article in ['robe', 'cape']:
    return 0.175
  elif article in ['short', 'jupe', 'jupon', 'legging', 'bermuda']:
    return 0.12
  elif article in ['pull', 'blouse', 'gilet', 'poncho']:
    return 0.28
  elif article in ['salopette', 'tablier', 'tunique']:
    return 0.35
  elif article in ['doudoune', 'veste', 'parka', 'cardigan', 'manteau', 'blouson', 'duffle', 'trench', 'caban']:
    return 0.5
  elif article in ['casquette', 'bonnet', 'chapeau', 'bob', 'chapka', 'beret']:
    return 0.07
  elif article in ['peignoir', 'nid', 'jean']:
    return 0.3
  elif article in ['body', 'boxer', 'culotte', 'barbotteuse', 'mariniere', 'calecon']:
    return 0.3
  elif article in ['foulard', 'snood', 'noeud', 'gants', 'headband', 'moufles', 'fichu', 'cravate', 'collant', 'echarpe']:
    return 0.3
  else: # lots
    return 0.6

df_composes['poids'] = df_composes.apply(get_weight, axis=1)

In [ ]:
# multiplions chaque float de chaque matière par le poids du vêtement pour obtenir des quantités en volumes mesurables et donc comparables 
to_multiply = ['coton', 'polyester', 'élasthanne', 'polyamide', 'viscose', 'acrylique', 'laine', 'lin']
df_composes[to_multiply] = df_composes[to_multiply].multiply(df_composes["poids"], axis="index").round(2)

In [ ]:
# dataframe final
df_composes

,composes,collection,articles,coton,polyester,élasthanne,polyamide,viscose,acrylique,laine,lin,quantite,poids
0,"[98% coton, 2% élasthanne]",bébé,sweat,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.150
1,[100% coton],bébé,lot,0.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.600
2,[100% coton],bébé,body,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.300
3,[100% coton],bébé,lot,0.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.600
4,[100% coton],bébé,lot,0.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4672,[100% coton],garçon,pyjama,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.150
4673,[100% acrylique],garçon,moufles,0.00,0.0,0.0,0.0,0.0,0.3,0.0,0.0,unique,0.300
4674,"[100% coton, 100% coton]",garçon,polo,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.150
4675,"[100% coton, 100% coton]",garçon,pyjama,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.150


In [ ]:
uploaded = files.upload()
path = "df_composes_final.csv"
df_composes_final = pd.read_csv(path)

In [ ]:
# manipulons et visualisons à partir de ce dataframe
poids = df_composes_final[['coton', 'polyester', 'élasthanne', 'polyamide', 'viscose', 'acrylique', 'laine', 'lin']].sum().to_frame()
poids['pourcentages'] = poids / poids.sum() * 100
poids

,0,pourcentages
coton,879.76,70.685028
polyester,212.25,17.053398
élasthanne,11.99,0.963346
polyamide,62.31,5.006347
viscose,27.87,2.239238
acrylique,38.63,3.103759
laine,6.45,0.518230
lin,5.36,0.430654


In [ ]:
# effectuons un pie chart
fig_poids_tot = go.Figure(data=[go.Pie(labels=poids['pourcentages'].index, values=poids['pourcentages'].values, hole=.5)])
fig_poids_tot.update_layout(height=500, width=1200, title_text='Répartition des volumes de matières en fonction du poids total', piecolorway=['#b3e2cd', '#fdccac', '#cbd5e8', '#f4cae4', '#e6f5c9', '#fff2ae', '#f1e2cc', '#cccccc'])
fig_poids_tot.show()

Nous voyons à quel point le **coton** est prédominant dans les collections de la marque.

In [ ]:
# visualisons la répartition par collection
by_collection = df_composes_final.groupby('collection')[['coton', 'polyester', 'élasthanne', 'polyamide', 'viscose', 'acrylique', 'laine', 'lin']].sum()
by_collection

,coton,polyester,élasthanne,polyamide,viscose,acrylique,laine,lin
collection,,,,,,,,
bébé,378.13,67.24,3.22,19.51,8.27,5.65,3.86,1.85
fille,305.63,80.76,5.83,24.52,17.49,19.27,1.90,0.36
garçon,196.00,64.25,2.94,18.28,2.11,13.71,0.69,3.15


In [ ]:
# calculons les poids totaux pour chaque collection
poids_total = by_collection.sum(axis=1)

# divisons chaque valeur par le poids total de la collection correspondante, puis multiplions par 100
df_pct = (by_collection.div(poids_total, axis=0) * 100).round(1)
df_pct

,coton,polyester,élasthanne,polyamide,viscose,acrylique,laine,lin
collection,,,,,,,,
bébé,77.5,13.8,0.7,4.0,1.7,1.2,0.8,0.4
fille,67.1,17.7,1.3,5.4,3.8,4.2,0.4,0.1
garçon,65.1,21.3,1.0,6.1,0.7,4.6,0.2,1.0


In [ ]:
# réalisons un stacked_bar pour représenter les proportions
trace_coton = go.Bar(name='Coton', x=df_pct.index, y=df_pct["coton"], text=df_pct["coton"], textposition='inside', marker_color='#b3e2cd')
trace_polyester = go.Bar(name='Polyester', x=df_pct.index, y=df_pct["polyester"], text=df_pct["polyester"], textposition='inside', marker_color='#fdccac')
trace_élasthanne = go.Bar(name='Elasthanne', x=df_pct.index, y=df_pct["élasthanne"], text=df_pct["élasthanne"], textposition='inside', marker_color='#cbd5e8')
trace_polyamide = go.Bar(name='Polyamide', x=df_pct.index, y=df_pct["polyamide"], text=df_pct["polyamide"], textposition='inside', marker_color='#f4cae4')
trace_viscose = go.Bar(name='Viscose', x=df_pct.index, y=df_pct["viscose"], text=df_pct["viscose"], textposition='inside', marker_color='#e6f5c9')
trace_acrylique = go.Bar(name='Acrylique', x=df_pct.index, y=df_pct["acrylique"], text=df_pct["acrylique"], textposition='inside', marker_color='#fff2ae')
trace_laine = go.Bar(name='Laine', x=df_pct.index, y=df_pct["laine"], text=df_pct["laine"], textposition='inside', marker_color='#f1e2cc')
trace_lin = go.Bar(name='Lin', x=df_pct.index, y=df_pct["lin"], text=df_pct["lin"], textposition='inside', marker_color='#cccccc')

stacked_bar_collec = go.Figure(data=[trace_coton, trace_polyester, trace_élasthanne, trace_polyamide, trace_viscose, trace_acrylique, trace_laine, trace_lin])
stacked_bar_collec.update_layout(title="Répartition des volumes de matières pour chaque collection", barmode='stack', plot_bgcolor="rgba(0,0,0,0)", yaxis=dict(showgrid=True, gridcolor='lightgray', range=[0, 100], tickformat='.%'), yaxis_title="% par rapport au poids total")
stacked_bar_collec.show()

Nous pouvons constater qu'il y a plus de coton dans les vêtements bébés comparativement aux collections filles et garçons. Cela peut s'expliquer par une attention portée par la marque aux spécificités des bébés (fragilité, sensibilité accrue). En revanche, on observe peu de différences entre les collections filles et garçons. Si l'on se réfère aux 3 catégories de vêtements, la collection bébés présente une majorité de matières naturelles (**78.7%**), puis viennent les matières synthétiques (**19.7%**) et enfin les matières artificielles (**1.7%**). Pour les collections filles et garçons confondues, viennent en premier lieu les matières naturelles également (**66.8%** en moyenne), puis synthétiques (**30.7%** en moyenne) et artificielles (**2.75%** en moyenne).

###**Détermination de notre indicateur : le score de toxicité**

Nous explicitons ici la démarche que nous avons adoptée pour obtenir un score de toxicité. Celui-ci a été obtenu à partir de l'évaluation de chaque matière sur une échelle de Likert, allant de **0 (pas du tout toxique)** à **5 (extrêmement toxique)**, puis a été attribué à chaque vêtement à partir d'un algorithme.</br>
Concernant les **matières synthétiques**, des sources nous apprennent que **70% des fibres synthétiques produites dans le monde proviennent du pétrole** ; par exemple, pour produire **1 kg de polyester, 1.5 kg de pétrole** serait nécessaire [[10]](https://www.wedressfair.fr/blog/c-est-quoi-le-probleme-des-fibres-synthetiques). Or, nous avons que le pétrole n'est pas renouvelable et est très polluant.</br> 
Pour être synthétisé puis teint, le polyester nécessite plusieurs produits chimiques potentiellement néfastes pour la santé comme la **soude caustique** (hydroxide de sodium) et l'**hydrosulfite de sodium** qui libère un gaz toxique (le dioxyde de soufre) lorsqu’il est en contact avec l’eau [[11]](https://maisonbrunon.fr/danger-du-polyester). Ces produits sont non seulement toxiques pour les personnes qui fabriquent les vêtements (et nous pouvons faire un rapprochement avec notre partie 1), mais également pour ceux qui les portent, car la matière est en contact permanent avec la peau. Selon cette même source, porter des sous-vêtements en polyester pourrait perturber le système endocrinien (mycoses, humidité, électricité statique...) Nous rappelons ici que les enfants sont plus vulnérables que tout autre personne du fait de son système immunitaire immature et en construction. Pour ces raisons, nous attribuerons un **score de 5** au polyester.</br>
**Le polyamide ou nylon**, représentait **9%** des fibres synthétiques utilisées dans le textile en 2011 [[12]](https://dreamact.eu/fr/blog/article/288/nylon-polyamide?utm_source=ART241_ART288). Utilisé notamment pour fabriquer les collants, c'est une matière également dérivée du pétrole, extrêmement polluante et qui a besoin de composés chimiques pour être synthétisée. Parmi eux, on trouve le **formaldéhyde** [[13]](https://dreamact.eu/fr/blog/article/241/matieres-ecologiques), un gaz volatile nocif entraînant des risques d'allergies et de pathologies respiratoires, et supposé cancérigène [[14]](https://www.asef-asso.fr/production/les-vetements-quand-les-toxiques-se-cachent-la-synthese-de-lasef/). Son seuil est limité à 20 ppm par vêtement en France. Par conséquent, nous attribuerons un **score de 5** au polyamide.</br>
**L'acrylique** est une autre matière synthétique utilisée par exemple pour les fausses fourrures, qui constitue **3.5%** des vêtements en synthétiques produits chaque année, nécessitant des produits chimiques tels que le **propylène**, l'**ammoniaque**, le **cyanure d’hydrogène** (un gaz très toxique pour l'être humain) [[13]](https://dreamact.eu/fr/blog/article/241/matieres-ecologiques). Par ailleurs, l'acrylique est fabriquée à partir de l'**acrylonitrile**, molécule ayant été révélée comme toxique et cancérigène chez le rat et l'humain [[15]](https://dreamact.eu/fr/blog/article/289/acrylique-polyacrylique-textile?utm_source=ART241_ART289). Nous mettrons également 5 comme score de toxicité pour cette matière.</br>
**L'élasthanne** est constitué à partir de polyuréthane et de PFC. Les PFC sont des **composés perfluorés** usités pour leurs qualités imperméables et élastiques, idéales pour les vêtements de sport. Or, ces produits sont des perturbateurs endocriniens et sont soupçonnés d'être cancérigènes [[14]](https://www.asef-asso.fr/production/les-vetements-quand-les-toxiques-se-cachent-la-synthese-de-lasef/) [[13]](https://dreamact.eu/fr/blog/article/241/matieres-ecologiques). Nous attribuerons également un **score de 5** à l'élasthanne pour ces raisons.</br></br>
Concernant les **matières artificielles**, la matière présente dans notre étude est la **viscose**, imitation de la soie, et réalisée à partir de cellulose de bambou. A l'origine le bambou est écologique mais le procédé de fabrication de la fibre nécessite de la **soude caustique**, du **disulfure de carbone**, des **sulfates** et de l'**acide sulfurique**, et est donc très toxique pour les travailleurs, les consommateurs et la nature [[13]](https://dreamact.eu/fr/blog/article/241/matieres-ecologiques).</br></br>
Concernant enfin les **matières naturelles**, celles-ci ne nécessitent pas de produits chimiques de synthèse. Nous avons ici le **lin** qui est considéré comme très écologique du fait de ses faibles besoins en eau et en pesticides, nous attribuerons donc un **score de 0** au lin, qui est par ailleurs une matière locale et durable.</br>
La **laine** est présente également dans notre étude. Elle ne demande aucun produit chimique et n'est donc aucunement toxique c'est pourquoi nous lui attribuons un **score de 0** dans le cadre de l'indicateur qui nous occupe. Toutefois, nous souhaitons préciser que la production de laine se fait dans des conditions attroces pour les bêtes qui sont malmenées et maltraitées pour l'obtenir [[13]](https://dreamact.eu/fr/blog/article/241/matieres-ecologiques).</br>
Enfin, citons le coton qui est la matière la plus représentée dans notre dataset. Matière naturelle ne nécessitant pas de produits de synthèse, mais décriée du fait de son gourmandise en eau, ainsi que l'utilisation des engrais et pesticides également néfastes pour la santé [[16]](https://www.cousubio.com/blog/technique-textile/fibres-textiles-naturelles-synthetiques-que-choisir). Nous mettrons donc au coton un **score de 2** du fait de ces critères. 

In [ ]:
# calculons le score de toxicité pour chaque vêtement et stockons-le dans une nouvelle colonne
# à noter : nous avons défini la fonction de telle sorte que si la colonne des composés comporte plus de 2 matières, nous retenons la 1re dont le % est systématiquement le plus important
# nous partons en effet du principe que cette matière est dominante dans le vêtement
toxicity_score = {
    'coton': 2,
    'polyester': 5,
    'élasthanne': 5,
    'polyamide': 5,
    'viscose': 5,
    'acrylique': 5,
    'laine': 0,
    'lin': 0
}

def calculate_toxicity(composition):
    score = 0
    for key in toxicity_score:
        if key in composition.split(',')[0]:
            score = toxicity_score[key]
            break
    return score

df_composes_final['toxicity_score'] = df_composes_final['composes'].apply(calculate_toxicity)

In [ ]:
df_composes_final.head()

,Unnamed: 0,composes,collection,articles,coton,polyester,élasthanne,polyamide,viscose,acrylique,laine,lin,quantite,poids,toxicity_score
0,0,"['98% coton', '2% élasthanne']",bébé,sweat,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.15,2
1,1,['100% coton'],bébé,lot,0.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.60,2
2,2,['100% coton'],bébé,body,0.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.30,2
3,3,['100% coton'],bébé,lot,0.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.60,2
4,4,['100% coton'],bébé,lot,0.60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unique,0.60,2


In [ ]:
# visualisons sur un pie chart la répartition des scores de toxicité attribués
fig_toxicity = go.Figure(data=[go.Pie(labels=df_composes_final['toxicity_score'].value_counts().index, values=df_composes_final['toxicity_score'].value_counts(), hole=.5)])
fig_toxicity.update_layout(height=500, width=1200, title_text='Répartition des scores de toxicité attribués', piecolorway=['#b3e2cd', '#fdccac', '#cbd5e8'])
fig_toxicity.show()

Nous observons une grande **majorité de scores à 2**, ce qui correspond au coton, cela étant dû au fait que nous avons programmé notre fonction pour ne retenir que la matière principale du vêtement, c'est-à-dire la matière placée en 1er dans la liste des compositions, présentant le plus haut pourcentage. La plupart du temps, il s'agit du coton. Voyons ce qu'il en est des moyennes de ces scores sur l'ensemble des vêtements, et sur chaque collection.

In [ ]:
# moyenne de tous les scores
df_composes_final['toxicity_score'].mean()

2.3890058479532166

In [ ]:
# moyennes sur chaque collection
df_composes_final.groupby('collection')['toxicity_score'].mean().round(1)

collection
bébé      2.2
fille     2.5
garçon    2.4
Name: toxicity_score, dtype: float64

In [ ]:
# visualisons tous ces scores sur un graphique à bulles
collections = ['bébé', 'fille', 'garçon']
values = [2.2, 2.5, 2.4]
mean_value = np.mean(values)

trace = go.Scatter(x=collections, y=values, mode='markers+text', text=values, textposition='top center', showlegend=False, marker=dict(size=300*values, sizemode='area', sizeref=0.005, color=values, colorscale='Viridis', showscale=True))
mean_trace = go.Scatter(x=collections, y=[mean_value]*len(collections), mode='lines', line=dict(color='red', width=2, dash='dash'), showlegend=False, hoverinfo='skip')
layout = go.Layout(title='Scores de toxicité selon la collection et comparativement à la moyenne globale', xaxis=dict(title='Collection'), yaxis=dict(title='Score'), plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
fig = go.Figure(data=[trace, mean_trace], layout=layout)
fig.add_annotation(x=0, y=2.39, text='Moyenne = {:.2f}'.format(mean_value), showarrow=False, font=dict(size=14, color='red'))
fig.show()

Ce graphique nous montre des scores moyens proches de 2.5, la moyenne de nos scores possibles (0 à 5). La note la plus basse étant celle des bébés (**2.2**), puis celle des garçons (**2.4**) et enfin des filles (**2.5**). La moyenne globale, toutes collections confondues, est de **2.37**. Selon les critères que nous avons fixés, nous pouvons en conclure que la marque Vertbaudet se situe dans une moyenne. **Il serait intéressant, bien évidemment, d'effectuer un comparatif avec d'autres marques, établi selon les mêmes critères. De manière absolue, il est impossible de se prononcer sur la qualité supérieure ou inférieure de cette marque en terme de sécurité du vêtement. Mais comme nous allons le voir dans la partie 3, ce calcul peut servir à alimenter des modèles servant de repères aux consommateurs dans leurs décisions d'achat.**</br></br>
Précisons également que pour toutes ces matières nous ne relevons ici que les impacts sur la santé, mais nous pourrions également nous étendre sur les nombreux impacts environnementaux (microparticules rejetées dans les eaux usées, émissions de gaz à effet de serre, consommation en eau pour la fabrication et le lavage, gaspillage et rejet des matières dans la nature, etc.) Outre les conditions de travail dans les usines de fabrication, nous attirons également l'attention, comme nous l'avons mentionné, sur la production de laine, maltraitante pour les animaux.
**Nous allons à présent concentrer notre dernière partie sur des solutions possibles à toutes ces constatations, à partir de l'utilisation de l'Intelligence Artificielle et de ses outils**.

# **Références du notebook**

[1] https://greenly.earth/fr-fr/blog/actualites-ecologie/empreinte-carbone-comparatif-transports

[2] https://consommerresponsable.com/empreinte-carbone-mode-de-transport-plus-polluant/

[3] https://www.franceindustrie.org/wp-franceindustrie/wp-content/uploads/2021/02/Synthese-Etude-UIT-Empreinte-carbone-280121.pdf

[4] https://engagements.decathlon.fr/pourquoi-faut-il-tant-deau-pour-fabriquer-un-vetement

[5] https://particuliers.engie.fr/economies-energie/conseils-economies-energie/conseils-calcul-consommation/comment-calculer-la-consommation-de-votre-lave-linge.html

[6] https://www.ubaldi.com/guides/lave-linge/caracteristiques-principales--gup208.php

[7] https://www.fournisseurs-electricite.com/guides/environnement/bilan-carbone/voiture

[8] https://www.notre-environnement.gouv.fr/actualites/breves/article/que-represente-l-empreinte-carbone-annuelle-d-un-francais

[9] https://www.terre-du-futur.fr/combien-de-co2-absorbe-un-arbre/

[10] https://www.wedressfair.fr/blog/c-est-quoi-le-probleme-des-fibres-synthetiques

[11] https://maisonbrunon.fr/danger-du-polyester

[12] https://dreamact.eu/fr/blog/article/288/nylon-polyamide?utm_source=ART241_ART288

[13] https://dreamact.eu/fr/blog/article/241/matieres-ecologiques

[14] https://www.asef-asso.fr/production/les-vetements-quand-les-toxiques-se-cachent-la-synthese-de-lasef/

[15] https://dreamact.eu/fr/blog/article/289/acrylique-polyacrylique-textile?utm_source=ART241_ART289

[16] https://www.cousubio.com/blog/technique-textile/fibres-textiles-naturelles-synthetiques-que-choisir